Similarity metrics for segmentation quality

## Non-labeled dataset

### ATLAS vs DL

Allignment

In [ ]:
import nibabel as nb
from pathlib import Path
import numpy as np
import glob, os

reg_dir = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset_nifti_reg_2/'
dl_dir = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/3D_multilabel/experiment_0/test_orig_reg-cropped_non-labeled'
output_path = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/3D_multilabel/experiment_0/test_orig_reg-cropped_non-labeled_allign'
if not os.path.exists(output_path):
    os.makedirs(output_path)

i=0
for folder1 in sorted(os.listdir(reg_dir)):

    # Name subject
    name_subject = folder1
    print(f'subject: {name_subject}')

    # Load t1_aux (to allign the other)
    t1_aux = nb.load(f'{reg_dir}{folder1}/labels.nii.gz')

    # Load t1 image (not alligned)
    t1 = nb.load(f'{dl_dir}{folder1}_cropped_seg_res.nii.gz')

    # Copy affine from t1_aux to t1
    nii = nb.Nifti1Image(t1.dataobj, t1_aux.affine, t1.header)
    nii.to_filename(f'{output_path}{name_subject}_cropped_seg_res_allign.nii.gz')
    
    # Dealing with files in that folder
    # for f in glob.glob(base_dir+folder1+'/input/'+folder1+'_T1_oriented_hdr.nii.gz'):
    #     os.remove(f)

    i+=1
    # if (i==1):
    #     break

Compute metrics

In [ ]:
import SimpleITK as sitk
import numpy as np
import pandas as pd
import os
import nibabel as nb

# nDSC (normalized DSC)
def dice_norm_metric(ground_truth, predictions):
    '''
    For a single example returns DSC_norm, fpr, fnr
    '''

    # Reference for normalized DSC
    r = 0.001 # It should be 1/N*(np.sum(voxels_label[i])/np.sum(voxels_image[i])) i belonging to training set
    # Cast to float32 type
    gt = ground_truth.astype("float32")
    seg = predictions.astype("float32")
    im_sum = np.sum(seg) + np.sum(gt)
    if im_sum == 0:
        return 1.0, 1.0, 1.0
    else:
        if np.sum(gt) == 0:
            k = 1.0
        else:
            k = (1 - r) * np.sum(gt) / (r * (len(gt.flatten()) - np.sum(gt)))
        tp = np.sum(seg[gt == 1])
        fp = np.sum(seg[gt == 0])
        fn = np.sum(gt[seg == 0])
        fp_scaled = k * fp
        dsc_norm = 2 * tp / (fp_scaled + 2 * tp + fn)

        fpr = fp / (len(gt.flatten()) - np.sum(gt))
        if np.sum(gt) == 0:
            fnr = 1.0
        else:
            fnr = fn / np.sum(gt)
        return dsc_norm # fpr, fnr

# Paths - segmentation results
reg_dir = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset_nifti_reg_2/'
dl_dir = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/3D_multilabel/experiment_0/test_orig_reg-cropped_non-labeled_allign'

# List of volumes for each individual label and subject
len_path = len([elem for elem in os.listdir(reg_dir)]) # elements in gt path = number of subjects
name_subject = [None]*len_path

# Save values in an array
# All labels
val_dsc             = np.zeros(len_path)
# val_hau             = np.zeros(len(rest_subjects))
val_hau_avg         = np.zeros(len_path)
val_vol             = np.zeros(len_path)
val_ndsc            = np.zeros(len_path)
# Lens
val_dsc_lens        = np.zeros(len_path)
val_hau_avg_lens    = np.zeros(len_path)
val_vol_lens        = np.zeros(len_path)
val_ndsc_lens       = np.zeros(len_path)
# Globe
val_dsc_globe       = np.zeros(len_path)
val_hau_avg_globe   = np.zeros(len_path)
val_vol_globe       = np.zeros(len_path)
val_ndsc_globe      = np.zeros(len_path)
# Optic nerve  
val_dsc_nerve       = np.zeros(len_path)
val_hau_avg_nerve   = np.zeros(len_path)
val_vol_nerve       = np.zeros(len_path)
val_ndsc_nerve      = np.zeros(len_path)
# Intraconal fat
val_dsc_int_fat     = np.zeros(len_path)
val_hau_avg_int_fat = np.zeros(len_path)
val_vol_int_fat     = np.zeros(len_path)
val_ndsc_int_fat    = np.zeros(len_path)
# Extraconal fat
val_dsc_ext_fat     = np.zeros(len_path)
val_hau_avg_ext_fat = np.zeros(len_path)
val_vol_ext_fat     = np.zeros(len_path)
val_ndsc_ext_fat    = np.zeros(len_path)
# Lateral rectus muscle
val_dsc_lat_mus     = np.zeros(len_path)
val_hau_avg_lat_mus = np.zeros(len_path)
val_vol_lat_mus     = np.zeros(len_path)
val_ndsc_lat_mus    = np.zeros(len_path)
# Medial rectus musclenp.zeros(len_path)
val_dsc_med_mus     = np.zeros(len_path)
val_hau_avg_med_mus = np.zeros(len_path)
val_vol_med_mus     = np.zeros(len_path)
val_ndsc_med_mus    = np.zeros(len_path)
# Inferior rectus muscle
val_dsc_inf_mus     = np.zeros(len_path)
val_hau_avg_inf_mus = np.zeros(len_path)
val_vol_inf_mus     = np.zeros(len_path)
val_ndsc_inf_mus    = np.zeros(len_path)
# Superior rectus muscnp.zeros(len_path)
val_dsc_sup_mus     = np.zeros(len_path)
val_hau_avg_sup_mus = np.zeros(len_path)
val_vol_sup_mus     = np.zeros(len_path)
val_ndsc_sup_mus    = np.zeros(len_path)
    
reader = sitk.ImageFileReader()

i=0
for folder1 in sorted(os.listdir(reg_dir)):

    # Name subject
    name_subject[i] = str(folder1)
    print(f'subject: {name_subject[i]}')
    
    # ATLAS-based registration results
    reg_lab_path = f'{reg_dir}{folder1}/labels.nii.gz'
    reader.SetFileName(reg_lab_path)
    reg_lab_sitk = sitk.Cast(reader.Execute(), sitk.sitkUInt8)
    reg_lab_arr = sitk.GetArrayFromImage(reg_lab_sitk)

    # Deep Learning results
    dl_lab_path = f'{dl_dir}{folder1}_cropped_seg_res_allign.nii.gz'
    reader.SetFileName(dl_lab_path)
    dl_lab_sitk = sitk.Cast(reader.Execute(), sitk.sitkUInt8)
    dl_lab_arr = sitk.GetArrayFromImage(dl_lab_sitk)
    
    # Image size
    im_lab_size = reg_lab_arr.shape[0]*reg_lab_arr.shape[1]*reg_lab_arr.shape[2]

    # LENS
    # Measures Image Filter 
    overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
    overlap_measures_filter.Execute(reg_lab_sitk==1, dl_lab_sitk==1)
    # DSC
    dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
    val_dsc_lens[i] = dsc
    # Volume
    vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
    val_vol_lens[i] = vol
    # Hausdorff distance
    if np.count_nonzero(reg_lab_arr==1)==0 or np.count_nonzero(dl_lab_arr==1)==0: val_hau_avg_lens[i] = 100
    else :
        hausdorf = sitk.HausdorffDistanceImageFilter()
        hausdorf.Execute(reg_lab_sitk==1, dl_lab_sitk==1)
        hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
        val_hau_avg_lens[i] = hausdorf_distance_avg
    # nDSC
    nDSC = dice_norm_metric(reg_lab_arr==1, dl_lab_arr==1)
    val_ndsc_lens[i] = nDSC
    
    # GLOBE EX LENS
    # Measures Image Filter 
    overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
    overlap_measures_filter.Execute(reg_lab_sitk==2, dl_lab_sitk==2)
    # DSC
    dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
    val_dsc_globe[i] = dsc
    # Volume
    vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
    val_vol_globe[i] = vol
    # Hausdorff distance
    hausdorf = sitk.HausdorffDistanceImageFilter()
    hausdorf.Execute(reg_lab_sitk==2, dl_lab_sitk==2)
    hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
    val_hau_avg_globe[i] = hausdorf_distance_avg
    # nDSC
    nDSC = dice_norm_metric(reg_lab_arr==2, dl_lab_arr==2)
    val_ndsc_globe[i] = nDSC

    # OPTIC NERVE
    # Measures Image Filter 
    overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
    overlap_measures_filter.Execute(reg_lab_sitk==3, dl_lab_sitk==3)
    # DSC
    dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
    val_dsc_nerve[i] = dsc
    # Volume
    vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
    val_vol_nerve[i] = vol
    # Hausdorff distance
    hausdorf = sitk.HausdorffDistanceImageFilter()
    hausdorf.Execute(reg_lab_sitk==3, dl_lab_sitk==3)
    hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
    val_hau_avg_nerve[i] = hausdorf_distance_avg
    # nDSC
    nDSC = dice_norm_metric(reg_lab_arr==3, dl_lab_arr==3)
    val_ndsc_nerve[i] = nDSC

    # INTRACONAL FAT
    # Measures Image Filter 
    overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
    overlap_measures_filter.Execute(reg_lab_sitk==4, dl_lab_sitk==4)
    # DSC
    dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
    val_dsc_int_fat[i] = dsc
    # Volume
    vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
    val_vol_int_fat[i] = vol
    # Hausdorff distance
    hausdorf = sitk.HausdorffDistanceImageFilter()
    hausdorf.Execute(reg_lab_sitk==4, dl_lab_sitk==4)
    hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
    val_hau_avg_int_fat[i] = hausdorf_distance_avg
    # nDSC
    nDSC = dice_norm_metric(reg_lab_arr==4, dl_lab_arr==4)
    val_ndsc_int_fat[i] = nDSC

    # EXTRACONAL FAT
    # Measures Image Filter 
    overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
    overlap_measures_filter.Execute(reg_lab_sitk==5, dl_lab_sitk==5)
    # DSC
    dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
    val_dsc_ext_fat[i] = dsc
    # Volume
    vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
    val_vol_ext_fat[i] = vol
    # Hausdorff distance
    hausdorf = sitk.HausdorffDistanceImageFilter()
    hausdorf.Execute(reg_lab_sitk==5, dl_lab_sitk==5)
    hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
    val_hau_avg_ext_fat[i] = hausdorf_distance_avg
    # nDSC
    nDSC = dice_norm_metric(reg_lab_arr==5, dl_lab_arr==5)
    val_ndsc_ext_fat[i] = nDSC

    # LATERAL RECTUS MUSCLE
    # Measures Image Filter 
    overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
    overlap_measures_filter.Execute(reg_lab_sitk==6, dl_lab_sitk==6)
    # DSC
    dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
    val_dsc_lat_mus[i] = dsc
    # Volume
    vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
    val_vol_lat_mus[i] = vol
    # Hausdorff distance
    hausdorf = sitk.HausdorffDistanceImageFilter()
    hausdorf.Execute(reg_lab_sitk==6, dl_lab_sitk==6)
    hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
    val_hau_avg_lat_mus[i] = hausdorf_distance_avg
    # nDSC
    nDSC = dice_norm_metric(reg_lab_arr==6, dl_lab_arr==6)
    val_ndsc_lat_mus[i] = nDSC

    # MEDIAL RECTUS MUSCLE
    # Measures Image Filter 
    overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
    overlap_measures_filter.Execute(reg_lab_sitk==7, dl_lab_sitk==7)
    # DSC
    dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
    val_dsc_med_mus[i] = dsc
    # Volume
    vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
    val_vol_med_mus[i] = vol
    # Hausdorff distance
    if np.count_nonzero(reg_lab_arr==7)==0 or np.count_nonzero(dl_lab_arr==7)==0: val_hau_avg_lens[i] = 100
    else :
        hausdorf = sitk.HausdorffDistanceImageFilter()
        hausdorf.Execute(reg_lab_sitk==7, dl_lab_sitk==7)
        hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
        val_hau_avg_med_mus[i] = hausdorf_distance_avg
    # nDSC
    nDSC = dice_norm_metric(reg_lab_arr==7, dl_lab_arr==7)
    val_ndsc_med_mus[i] = nDSC

    # INFERIOR RECTUS MUSCLE
    # Measures Image Filter 
    overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
    overlap_measures_filter.Execute(reg_lab_sitk==8, dl_lab_sitk==8)
    # DSC
    dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
    val_dsc_inf_mus[i] = dsc
    # Volume
    vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
    val_vol_inf_mus[i] = vol
    # Hausdorff distance
    if np.count_nonzero(reg_lab_arr==8)==0 or np.count_nonzero(dl_lab_arr==8)==0: val_hau_avg_lens[i] = 100
    else :
        hausdorf = sitk.HausdorffDistanceImageFilter()
        hausdorf.Execute(reg_lab_sitk==8, dl_lab_sitk==8)
        hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
        val_hau_avg_inf_mus[i] = hausdorf_distance_avg
    # nDSC
    nDSC = dice_norm_metric(reg_lab_arr==8, dl_lab_arr==8)
    val_ndsc_inf_mus[i] = nDSC

    # SUPERIOR RECTUS MUSCLE
    # Measures Image Filter 
    overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
    overlap_measures_filter.Execute(reg_lab_sitk==9, dl_lab_sitk==9)
    # DSC
    dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
    val_dsc_sup_mus[i] = dsc
    # Volume
    vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
    val_vol_sup_mus[i] = vol
    # Hausdorff distance
    hausdorf = sitk.HausdorffDistanceImageFilter()
    hausdorf.Execute(reg_lab_sitk==9, dl_lab_sitk==9)
    hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
    val_hau_avg_sup_mus[i] = hausdorf_distance_avg
    # nDSCvol
    nDSC = dice_norm_metric(reg_lab_arr==9, dl_lab_arr==9)
    val_ndsc_sup_mus[i] = nDSC

    # ALL LABELS
    # DSC
    dsc = (val_dsc_lens[i]+val_dsc_globe[i]+val_dsc_nerve[i]+val_dsc_int_fat[i]+val_dsc_ext_fat[i]+val_dsc_lat_mus[i]+val_dsc_med_mus[i]+val_dsc_inf_mus[i]+val_dsc_sup_mus[i])/9
    val_dsc[i] = dsc
    # Volume
    vol = (val_vol_lens[i]+val_vol_globe[i]+val_vol_nerve[i]+val_vol_int_fat[i]+val_vol_ext_fat[i]+val_vol_lat_mus[i]+val_vol_med_mus[i]+val_vol_inf_mus[i]+val_vol_sup_mus[i])/9
    val_vol[i] = vol
    # Hausdorff distance
    hau_avg = (val_hau_avg_lens[i]+val_hau_avg_globe[i]+val_hau_avg_nerve[i]+val_hau_avg_int_fat[i]+val_hau_avg_ext_fat[i]+val_hau_avg_lat_mus[i]+val_hau_avg_med_mus[i]+val_hau_avg_inf_mus[i]+val_hau_avg_sup_mus[i])/9
    val_hau_avg[i] = hau_avg
    # nDSC
    nDSC = (val_ndsc_lens[i]+val_ndsc_globe[i]+val_ndsc_nerve[i]+val_ndsc_int_fat[i]+val_ndsc_ext_fat[i]+val_ndsc_lat_mus[i]+val_ndsc_med_mus[i]+val_ndsc_inf_mus[i]+val_ndsc_sup_mus[i])/9
    val_ndsc[i] = nDSC

    i+=1
    # if i==2:
    #     break

Save values to .csv

In [ ]:
import csv

metrics = [
    'Subject',  'DSC_all',     'Haus_avg_all',     'Volume_all',     'nDSC_all', 
                'DSC_lens',    'Haus_avg_lens',    'Volume_lens',    'nDSC_lens',  
                'DSC_globe',   'Haus_avg_globe',   'Volume_globe',   'nDSC_globe', 
                'DSC_nerve',   'Haus_avg_nerve',   'Volume_nerve',   'nDSC_nerve', 
                'DSC_int_fat', 'Haus_avg_int_fat', 'Volume_int_fat', 'nDSC_int_fat',
                'DSC_ext_fat', 'Haus_avg_ext_fat', 'Volume_ext_fat', 'nDSC_ext_fat',
                'DSC_lat_mus', 'Haus_avg_lat_mus', 'Volume_lat_mus', 'nDSC_lat_mus',
                'DSC_med_mus', 'Haus_avg_med_mus', 'Volume_med_mus', 'nDSC_med_mus',
                'DSC_inf_mus', 'Haus_avg_inf_mus', 'Volume_inf_mus', 'nDSC_inf_mus',
                'DSC_sup_mus', 'Haus_avg_sup_mus', 'Volume_sup_mus', 'nDSC_sup_mus'
]

vals = np.array([
    name_subject,   val_dsc,         val_hau_avg,         val_vol,         val_ndsc,        
                    val_dsc_lens,    val_hau_avg_lens,    val_vol_lens,    val_ndsc_lens,   
                    val_dsc_globe,   val_hau_avg_globe,   val_vol_globe,   val_ndsc_globe,  
                    val_dsc_nerve,   val_hau_avg_nerve,   val_vol_nerve,   val_ndsc_nerve,  
                    val_dsc_int_fat, val_hau_avg_int_fat, val_vol_int_fat, val_ndsc_int_fat,
                    val_dsc_ext_fat, val_hau_avg_ext_fat, val_vol_ext_fat, val_ndsc_ext_fat,
                    val_dsc_lat_mus, val_hau_avg_lat_mus, val_vol_lat_mus, val_ndsc_lat_mus,
                    val_dsc_med_mus, val_hau_avg_med_mus, val_vol_med_mus, val_ndsc_med_mus,
                    val_dsc_inf_mus, val_hau_avg_inf_mus, val_vol_inf_mus, val_ndsc_inf_mus,
                    val_dsc_sup_mus, val_hau_avg_sup_mus, val_vol_sup_mus, val_ndsc_sup_mus,
])

vals = vals.T

with open('/mnt/sda1/Repos/a-eye/Output/similarity/sim_reg_vs_dl.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow(metrics)
    writer.writerows(vals)

Read values from .csv

In [ ]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.pyplot import Line2D
import numpy as np

csv_file = pd.read_csv('/mnt/sda1/Repos/a-eye/Output/similarity/sim_reg_vs_dl.csv')

# Dataframes {DSC, nDSC, Volume (voxels)} separate labels for N=5 only
data_dsc = [csv_file['DSC_all'], csv_file['DSC_lens'], csv_file['DSC_globe'], csv_file['DSC_nerve'], csv_file['DSC_int_fat'], csv_file['DSC_ext_fat'], csv_file['DSC_lat_mus'], csv_file['DSC_med_mus'], csv_file['DSC_inf_mus'], csv_file['DSC_sup_mus']]
data_ndsc = [csv_file['nDSC_all'],  csv_file['nDSC_lens'], csv_file['nDSC_globe'], csv_file['nDSC_nerve'], csv_file['nDSC_int_fat'], csv_file['nDSC_ext_fat'], csv_file['nDSC_lat_mus'], csv_file['nDSC_med_mus'], csv_file['nDSC_inf_mus'], csv_file['nDSC_sup_mus']]
data_vol = [csv_file['Volume_all'], csv_file['Volume_lens'], csv_file['Volume_globe'], csv_file['Volume_nerve'], csv_file['Volume_int_fat'], csv_file['Volume_ext_fat'], csv_file['Volume_lat_mus'], csv_file['Volume_med_mus'], csv_file['Volume_inf_mus'], csv_file['Volume_sup_mus']]
data_haus = [csv_file['Haus_avg_all'], csv_file['Haus_avg_lens'], csv_file['Haus_avg_globe'], csv_file['Haus_avg_nerve'], csv_file['Haus_avg_int_fat'], csv_file['Haus_avg_ext_fat'], csv_file['Haus_avg_lat_mus'], csv_file['Haus_avg_med_mus'], csv_file['Haus_avg_inf_mus'], csv_file['Haus_avg_sup_mus']]

labels = ['lens', 'globe', 'nerve', 'intraconal fat', 'extraconal fat', 'lateral rectus muscle', 'medial rectus muscle', 'inferior rectus muscle', 'superior rectus muscle']
median = [np.around(np.median(x), 2) for x in data_dsc]
for i in range(len(labels)):
    print(labels[i], median[i])

# Figure
fig, axs = plt.subplots(3, figsize=(20,10), sharex=True)
fig.canvas.manager.set_window_title('Similarity metrics ATLAS vs DL')
fig.suptitle('Similarity metrics ATLAS vs DL')
fig.patch.set_facecolor('white')

# Set labels and titles
axs[0].set_title('DSC')
axs[0].set_ylim([0, 1])
axs[1].set_title('Hausdorff distance')
axs[1].set_ylim([0, 3])
axs[2].set_title('Volume difference')
axs[2].set_ylim([-2, 2])

# Outliers
flierprops = dict(markerfacecolor='0.9', markersize=4, linestyle='none')

# Boxplot & Swarmplot (points)
graph1 = sns.boxplot(data=data_dsc, ax=axs[0], flierprops=flierprops).set(ylabel="Value")
graph2 = sns.boxplot(data=data_haus, ax=axs[1], flierprops=flierprops).set(ylabel="Value")
graph3 = sns.boxplot(data=data_vol, ax=axs[2], flierprops=flierprops).set(ylabel="Value")
axs[2].set_xticklabels(['all','lens','globe','nerve','int_fat','ext_fat','lat_mus','med_mus','inf_mus','sup_mus'])
axs[2].tick_params(bottom=True)

plt.show

# Save figure
plt.savefig('/mnt/sda1/Repos/a-eye/Output/similarity/sim_reg_vs_dl.png', bbox_inches='tight')

### ATLAS vs nnUnet

Cropping foreground by bounding box

In [ ]:
import numpy as np
import nibabel as nib
import SimpleITK as sitk
import pandas as pd
import glob, os

bound = 15 # boundary for the bounding box (margins)

reg_dir = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset_nifti_reg/'
nnunet_dir = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet/nnUNet_inference/no_postprocessing/'
output_dir = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet/nnUNet_inference/no_postprocessing_cropped/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

i=0
for folder1 in sorted(glob.glob(nnunet_dir+'*.nii.gz')):

    subject = folder1.split('/')[-1].split('.')[0].split('_')[-1]
    print(f'subject = {subject}')
    # if subject == '2022160101206': # indent from here
    image_path_nnunet = f'{nnunet_dir}AEye_{subject}.nii.gz'
    image_path_atlas = f'{reg_dir}{subject}/labels.nii.gz'

    # Atlas image size
    atlas_image = sitk.ReadImage(image_path_atlas)
    atlas_image_x_size, atlas_image_y_size, atlas_image_z_size = atlas_image.GetSize()
    print(f"atlas_image_x_size = {atlas_image_x_size} | atlas_image_y_size = {atlas_image_y_size} | atlas_image_z_size = {atlas_image_z_size}")

    # nnUNet image size
    nnunet_image = sitk.ReadImage(image_path_nnunet)
    nnunet_image_x_size, nnunet_image_y_size, nnunet_image_z_size = nnunet_image.GetSize()
    print(f"image_x_size = {nnunet_image_x_size} | image_y_size = {nnunet_image_y_size} | image_z_size = {nnunet_image_z_size}")

    # Mask
    atlas_image_arr = sitk.GetArrayFromImage(atlas_image)
    if np.count_nonzero(atlas_image_arr > 0) == 0:
        print(f'Image {subject} is empty')
        continue
    all_segments_mask = atlas_image > 0

    # Bounding box
    lsif = sitk.LabelStatisticsImageFilter() # It requires intensity and label images
    lsif.Execute(atlas_image, all_segments_mask) # Mask! Where all the labels are 1!
    bounding_box = np.array(lsif.GetBoundingBox(1)) # GetBoundingBox(label)
    print(f"Bounding box:  {bounding_box}") # [xmin, xmax, ymin, ymax, zmin, zmax]

    # Expand bounding box
    bounding_box_expanded = bounding_box.copy()
    bounding_box_expanded[0::2] -= bound # even indexes
    bounding_box_expanded[1::2] += bound # odd indexes
    print(f"Expanded bounding box: {bounding_box_expanded}")

    # Limits
    if bounding_box_expanded[0] < 0: bounding_box_expanded[0] = 0
    if bounding_box_expanded[1] > nnunet_image_x_size: bounding_box_expanded[1] = nnunet_image_x_size
    if bounding_box_expanded[2] < 0: bounding_box_expanded[2] = 0
    if bounding_box_expanded[3] > nnunet_image_y_size: bounding_box_expanded[3] = nnunet_image_y_size
    if bounding_box_expanded[4] < 0: bounding_box_expanded[4] = 0
    if bounding_box_expanded[5] > nnunet_image_z_size: bounding_box_expanded[5] = nnunet_image_z_size
    print(f"Expanded bounding box after limits: {bounding_box_expanded}")

    # Crop
    print(f'New image size: image_x_size = {bounding_box_expanded[1]-bounding_box_expanded[0]} | image_y_size = {bounding_box_expanded[3]-bounding_box_expanded[2]} | image_z_size = {bounding_box_expanded[5]-bounding_box_expanded[4]}')
    image_crop = nnunet_image[int(bounding_box_expanded[0]):int(bounding_box_expanded[1]), # x
                    int(bounding_box_expanded[2]):int(bounding_box_expanded[3]), # y
                    int(bounding_box_expanded[4]):int(bounding_box_expanded[5])] # z
    # sitk.WriteImage(image_crop, f'{output_dir}{subject}_cropped.nii.gz')

    i+=1
    if (i==1):
        break

How many images are in bytes instead of kB?

In [ ]:
# number of files under 1kB in a specific folder

import os
import glob

path = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet/nnUNet_inference/no_postprocessing_cropped/'
files = glob.glob(path + '*.nii.gz')
bad_files = []

i=0
for file in sorted(files):
    if os.path.getsize(file) < 1000:
        subject = file.split('/')[-1].split('.')[0].split('_')[0]
        bad_files.append(subject)
        print(subject)
        i+=1

print(f'Number of images under 1kB: {i}')

In [ ]:
import pandas as pd

qc3_vol_nnunet = pd.read_excel('/home/jaimebarranco/Desktop/MRI-QC/qc3_vol_nnunet.xlsx')

# Check for the presence of each file in qc3_vol_nunet
matching_files = [file for file in bad_files if file in str(qc3_vol_nnunet['subject'].values)]

# difference between matching_files and bad_files
difference = [file for file in bad_files if file not in matching_files]
print(difference) # that file was already removed in qc1 (mriqc-learn)

Allignment

In [ ]:
import numpy as np
import nibabel as nb
import SimpleITK as sitk
import pandas as pd
import glob, os

reg_dir = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset_nifti_reg_2/'
nnunet_dir = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet/nnUNet_inference/no_postprocessing_cropped/'
output_dir = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet/nnUNet_inference/no_postprocessing_cropped_alligned/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

i=0
for folder1 in sorted(glob.glob(nnunet_dir+'*.nii.gz')):

    subject = folder1.split('/')[-1].split('.')[0].split('_')[0]
    print(f'subject = {subject}')
    # if subject == '2022160100236': # indent from here
    image_path_nnunet = f'{nnunet_dir}{subject}_cropped.nii.gz'
    image_path_atlas = f'{reg_dir}{subject}/labels.nii.gz'

    # Load t1_aux (to allign the other)
    t1_aux = nb.load(image_path_atlas)
    # print(f't1_aux.affine: \n{t1_aux.affine}')
    # print(f't1_aux.header: \n{t1_aux.header}')

    # Load t1 image (not alligned)
    t1 = nb.load(image_path_nnunet)
    # print(f't1.affine: \n{t1.affine}')
    # print(f't1.header: \n{t1.header}')

    # Copy affine from t1_aux to t1
    nii = nb.Nifti1Image(t1.dataobj, t1_aux.affine, t1_aux.header)
    nii.to_filename(f'{output_dir}{subject}_cropped_alligned.nii.gz')
    
    # Dealing with files in that folder
    # for f in glob.glob(base_dir+folder1+'/input/'+folder1+'_T1_oriented_hdr.nii.gz'):
    #     os.remove(f)

    i+=1
    # if (i==1):
    #     break

Compute sim. metrics

In [ ]:
import SimpleITK as sitk
import numpy as np
import pandas as pd
import os
import nibabel as nb

# nDSC (normalized DSC)
def dice_norm_metric(ground_truth, predictions):
    '''
    For a single example returns DSC_norm, fpr, fnr
    '''

    # Reference for normalized DSC
    r = 0.001 # It should be 1/N*(np.sum(voxels_label[i])/np.sum(voxels_image[i])) i belonging to training set
    # Cast to float32 type
    gt = ground_truth.astype("float32")
    seg = predictions.astype("float32")
    im_sum = np.sum(seg) + np.sum(gt)
    if im_sum == 0:
        return 1.0, 1.0, 1.0
    else:
        if np.sum(gt) == 0:
            k = 1.0
        else:
            k = (1 - r) * np.sum(gt) / (r * (len(gt.flatten()) - np.sum(gt)))
        tp = np.sum(seg[gt == 1])
        fp = np.sum(seg[gt == 0])
        fn = np.sum(gt[seg == 0])
        fp_scaled = k * fp
        dsc_norm = 2 * tp / (fp_scaled + 2 * tp + fn)

        fpr = fp / (len(gt.flatten()) - np.sum(gt))
        if np.sum(gt) == 0:
            fnr = 1.0
        else:
            fnr = fn / np.sum(gt)
        return dsc_norm # fpr, fnr

# Paths - segmentation results
reg_dir = '/mnt/sda1/Repos/a-eye/Data/SHIP_dataset/non_labeled_dataset_nifti_reg_2/'
nnunet_dir = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet/nnUNet_inference/no_postprocessing_cropped_alligned/'

# List of volumes for each individual label and subject
len_path = len([elem for elem in os.listdir(nnunet_dir)]) # elements in gt path = number of subjects
name_subject = [None]*len_path

# Save values in an array
# All labels
val_dsc             = np.zeros(len_path)
# val_hau             = np.zeros(len(rest_subjects))
val_hau_avg         = np.zeros(len_path)
val_vol             = np.zeros(len_path)
val_ndsc            = np.zeros(len_path)
# Lens
val_dsc_lens        = np.zeros(len_path)
val_hau_avg_lens    = np.zeros(len_path)
val_vol_lens        = np.zeros(len_path)
val_ndsc_lens       = np.zeros(len_path)
# Globe
val_dsc_globe       = np.zeros(len_path)
val_hau_avg_globe   = np.zeros(len_path)
val_vol_globe       = np.zeros(len_path)
val_ndsc_globe      = np.zeros(len_path)
# Optic nerve  
val_dsc_nerve       = np.zeros(len_path)
val_hau_avg_nerve   = np.zeros(len_path)
val_vol_nerve       = np.zeros(len_path)
val_ndsc_nerve      = np.zeros(len_path)
# Intraconal fat
val_dsc_int_fat     = np.zeros(len_path)
val_hau_avg_int_fat = np.zeros(len_path)
val_vol_int_fat     = np.zeros(len_path)
val_ndsc_int_fat    = np.zeros(len_path)
# Extraconal fat
val_dsc_ext_fat     = np.zeros(len_path)
val_hau_avg_ext_fat = np.zeros(len_path)
val_vol_ext_fat     = np.zeros(len_path)
val_ndsc_ext_fat    = np.zeros(len_path)
# Lateral rectus muscle
val_dsc_lat_mus     = np.zeros(len_path)
val_hau_avg_lat_mus = np.zeros(len_path)
val_vol_lat_mus     = np.zeros(len_path)
val_ndsc_lat_mus    = np.zeros(len_path)
# Medial rectus muscle
val_dsc_med_mus     = np.zeros(len_path)
val_hau_avg_med_mus = np.zeros(len_path)
val_vol_med_mus     = np.zeros(len_path)
val_ndsc_med_mus    = np.zeros(len_path)
# Inferior rectus muscle
val_dsc_inf_mus     = np.zeros(len_path)
val_hau_avg_inf_mus = np.zeros(len_path)
val_vol_inf_mus     = np.zeros(len_path)
val_ndsc_inf_mus    = np.zeros(len_path)
# Superior rectus muscle
val_dsc_sup_mus     = np.zeros(len_path)
val_hau_avg_sup_mus = np.zeros(len_path)
val_vol_sup_mus     = np.zeros(len_path)
val_ndsc_sup_mus    = np.zeros(len_path)

reader = sitk.ImageFileReader()

i=0
for file in sorted(os.listdir(nnunet_dir)):

    # Name subject
    name_subject[i] = file.split('_')[0]
    print(f'subject: {name_subject[i]}')

    # bad files
    # if name_subject[i] in bad_files:
    #     print('bad file')
    #     i+=1
    #     continue
    
    # ATLAS-based registration results
    reg_lab_path = f'{reg_dir}{name_subject[i]}/labels.nii.gz'
    reader.SetFileName(reg_lab_path)
    reg_lab_sitk = sitk.Cast(reader.Execute(), sitk.sitkUInt8)
    reg_lab_arr = sitk.GetArrayFromImage(reg_lab_sitk)

    # Deep Learning results
    dl_lab_path = f'{nnunet_dir}{name_subject[i]}_cropped_alligned.nii.gz'
    # check if the file exists
    if not os.path.exists(dl_lab_path):
        print('file does not exist')
        i+=1
        continue
    reader.SetFileName(dl_lab_path)
    dl_lab_sitk = sitk.Cast(reader.Execute(), sitk.sitkUInt8)
    dl_lab_arr = sitk.GetArrayFromImage(dl_lab_sitk)
    
    # Image size
    im_lab_size = reg_lab_arr.shape[0]*reg_lab_arr.shape[1]*reg_lab_arr.shape[2]

    # LENS
    if np.count_nonzero(reg_lab_arr==1)==0 or np.count_nonzero(dl_lab_arr==1)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==1, dl_lab_sitk==1)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_lens[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_lens[i] = vol
        # Hausdorff distance
        hausdorf = sitk.HausdorffDistanceImageFilter()
        hausdorf.Execute(reg_lab_sitk==1, dl_lab_sitk==1)
        hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
        val_hau_avg_lens[i] = hausdorf_distance_avg
        # nDSC
        nDSC = dice_norm_metric(reg_lab_arr==1, dl_lab_arr==1)
        val_ndsc_lens[i] = nDSC
    
    # GLOBE EX LENS
    if np.count_nonzero(reg_lab_arr==2)==0 or np.count_nonzero(dl_lab_arr==2)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter 
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==2, dl_lab_sitk==2)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_globe[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_globe[i] = vol
        # Hausdorff distance
        hausdorf = sitk.HausdorffDistanceImageFilter()
        hausdorf.Execute(reg_lab_sitk==2, dl_lab_sitk==2)
        hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
        val_hau_avg_globe[i] = hausdorf_distance_avg
        # nDSC
        nDSC = dice_norm_metric(reg_lab_arr==2, dl_lab_arr==2)
        val_ndsc_globe[i] = nDSC

    # OPTIC NERVE
    if np.count_nonzero(reg_lab_arr==3)==0 or np.count_nonzero(dl_lab_arr==3)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter 
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==3, dl_lab_sitk==3)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_nerve[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_nerve[i] = vol
        # Hausdorff distance
        hausdorf = sitk.HausdorffDistanceImageFilter()
        hausdorf.Execute(reg_lab_sitk==3, dl_lab_sitk==3)
        hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
        val_hau_avg_nerve[i] = hausdorf_distance_avg
        # nDSC
        nDSC = dice_norm_metric(reg_lab_arr==3, dl_lab_arr==3)
        val_ndsc_nerve[i] = nDSC

    # INTRACONAL FAT
    if np.count_nonzero(reg_lab_arr==4)==0 or np.count_nonzero(dl_lab_arr==4)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter 
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==4, dl_lab_sitk==4)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_int_fat[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_int_fat[i] = vol
        # Hausdorff distance
        hausdorf = sitk.HausdorffDistanceImageFilter()
        hausdorf.Execute(reg_lab_sitk==4, dl_lab_sitk==4)
        hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
        val_hau_avg_int_fat[i] = hausdorf_distance_avg
        # nDSC
        nDSC = dice_norm_metric(reg_lab_arr==4, dl_lab_arr==4)
        val_ndsc_int_fat[i] = nDSC

    # EXTRACONAL FAT
    if np.count_nonzero(reg_lab_arr==5)==0 or np.count_nonzero(dl_lab_arr==5)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter 
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==5, dl_lab_sitk==5)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_ext_fat[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_ext_fat[i] = vol
        # Hausdorff distance
        hausdorf = sitk.HausdorffDistanceImageFilter()
        hausdorf.Execute(reg_lab_sitk==5, dl_lab_sitk==5)
        hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
        val_hau_avg_ext_fat[i] = hausdorf_distance_avg
        # nDSC
        nDSC = dice_norm_metric(reg_lab_arr==5, dl_lab_arr==5)
        val_ndsc_ext_fat[i] = nDSC

    # LATERAL RECTUS MUSCLE
    if np.count_nonzero(reg_lab_arr==6)==0 or np.count_nonzero(dl_lab_arr==6)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==6, dl_lab_sitk==6)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_lat_mus[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_lat_mus[i] = vol
        # Hausdorff distance
        hausdorf = sitk.HausdorffDistanceImageFilter()
        hausdorf.Execute(reg_lab_sitk==6, dl_lab_sitk==6)
        hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
        val_hau_avg_lat_mus[i] = hausdorf_distance_avg
        # nDSC
        nDSC = dice_norm_metric(reg_lab_arr==6, dl_lab_arr==6)
        val_ndsc_lat_mus[i] = nDSC

    # MEDIAL RECTUS MUSCLE
    if np.count_nonzero(reg_lab_arr==7)==0 or np.count_nonzero(dl_lab_arr==7)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter 
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==7, dl_lab_sitk==7)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_med_mus[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_med_mus[i] = vol
        # Hausdorff distance
        if np.count_nonzero(reg_lab_arr==7)==0 or np.count_nonzero(dl_lab_arr==7)==0: val_hau_avg_lens[i] = 100
        else :
            hausdorf = sitk.HausdorffDistanceImageFilter()
            hausdorf.Execute(reg_lab_sitk==7, dl_lab_sitk==7)
            hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
            val_hau_avg_med_mus[i] = hausdorf_distance_avg
        # nDSC
        nDSC = dice_norm_metric(reg_lab_arr==7, dl_lab_arr==7)
        val_ndsc_med_mus[i] = nDSC

    # INFERIOR RECTUS MUSCLE
    if np.count_nonzero(reg_lab_arr==8)==0 or np.count_nonzero(dl_lab_arr==8)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==8, dl_lab_sitk==8)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_inf_mus[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_inf_mus[i] = vol
        # Hausdorff distance
        if np.count_nonzero(reg_lab_arr==8)==0 or np.count_nonzero(dl_lab_arr==8)==0: val_hau_avg_lens[i] = 100
        else :
            hausdorf = sitk.HausdorffDistanceImageFilter()
            hausdorf.Execute(reg_lab_sitk==8, dl_lab_sitk==8)
            hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
            val_hau_avg_inf_mus[i] = hausdorf_distance_avg
        # nDSC
        nDSC = dice_norm_metric(reg_lab_arr==8, dl_lab_arr==8)
        val_ndsc_inf_mus[i] = nDSC

    # SUPERIOR RECTUS MUSCLE
    if np.count_nonzero(reg_lab_arr==9)==0 or np.count_nonzero(dl_lab_arr==9)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==9, dl_lab_sitk==9)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_sup_mus[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_sup_mus[i] = vol
        # Hausdorff distance
        hausdorf = sitk.HausdorffDistanceImageFilter()
        hausdorf.Execute(reg_lab_sitk==9, dl_lab_sitk==9)
        hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
        val_hau_avg_sup_mus[i] = hausdorf_distance_avg
        # nDSCvol
        nDSC = dice_norm_metric(reg_lab_arr==9, dl_lab_arr==9)
        val_ndsc_sup_mus[i] = nDSC

    # ALL LABELS
    # DSC
    dsc = (val_dsc_lens[i]+val_dsc_globe[i]+val_dsc_nerve[i]+val_dsc_int_fat[i]+val_dsc_ext_fat[i]+val_dsc_lat_mus[i]+val_dsc_med_mus[i]+val_dsc_inf_mus[i]+val_dsc_sup_mus[i])/9
    val_dsc[i] = dsc
    # Volume
    vol = (val_vol_lens[i]+val_vol_globe[i]+val_vol_nerve[i]+val_vol_int_fat[i]+val_vol_ext_fat[i]+val_vol_lat_mus[i]+val_vol_med_mus[i]+val_vol_inf_mus[i]+val_vol_sup_mus[i])/9
    val_vol[i] = vol
    # Hausdorff distance
    hau_avg = (val_hau_avg_lens[i]+val_hau_avg_globe[i]+val_hau_avg_nerve[i]+val_hau_avg_int_fat[i]+val_hau_avg_ext_fat[i]+val_hau_avg_lat_mus[i]+val_hau_avg_med_mus[i]+val_hau_avg_inf_mus[i]+val_hau_avg_sup_mus[i])/9
    val_hau_avg[i] = hau_avg
    # nDSC
    nDSC = (val_ndsc_lens[i]+val_ndsc_globe[i]+val_ndsc_nerve[i]+val_ndsc_int_fat[i]+val_ndsc_ext_fat[i]+val_ndsc_lat_mus[i]+val_ndsc_med_mus[i]+val_ndsc_inf_mus[i]+val_ndsc_sup_mus[i])/9
    val_ndsc[i] = nDSC

    i+=1
    # if i==1:
    #     break

Save values to csv

In [ ]:
import csv

metrics = [
    'Subject',  'DSC_all',     'Haus_avg_all',     'Volume_all',     'nDSC_all', 
                'DSC_lens',    'Haus_avg_lens',    'Volume_lens',    'nDSC_lens',  
                'DSC_globe',   'Haus_avg_globe',   'Volume_globe',   'nDSC_globe', 
                'DSC_nerve',   'Haus_avg_nerve',   'Volume_nerve',   'nDSC_nerve', 
                'DSC_int_fat', 'Haus_avg_int_fat', 'Volume_int_fat', 'nDSC_int_fat',
                'DSC_ext_fat', 'Haus_avg_ext_fat', 'Volume_ext_fat', 'nDSC_ext_fat',
                'DSC_lat_mus', 'Haus_avg_lat_mus', 'Volume_lat_mus', 'nDSC_lat_mus',
                'DSC_med_mus', 'Haus_avg_med_mus', 'Volume_med_mus', 'nDSC_med_mus',
                'DSC_inf_mus', 'Haus_avg_inf_mus', 'Volume_inf_mus', 'nDSC_inf_mus',
                'DSC_sup_mus', 'Haus_avg_sup_mus', 'Volume_sup_mus', 'nDSC_sup_mus'
]

vals = np.array([
    name_subject,   val_dsc,         val_hau_avg,         val_vol,         val_ndsc,        
                    val_dsc_lens,    val_hau_avg_lens,    val_vol_lens,    val_ndsc_lens,   
                    val_dsc_globe,   val_hau_avg_globe,   val_vol_globe,   val_ndsc_globe,  
                    val_dsc_nerve,   val_hau_avg_nerve,   val_vol_nerve,   val_ndsc_nerve,  
                    val_dsc_int_fat, val_hau_avg_int_fat, val_vol_int_fat, val_ndsc_int_fat,
                    val_dsc_ext_fat, val_hau_avg_ext_fat, val_vol_ext_fat, val_ndsc_ext_fat,
                    val_dsc_lat_mus, val_hau_avg_lat_mus, val_vol_lat_mus, val_ndsc_lat_mus,
                    val_dsc_med_mus, val_hau_avg_med_mus, val_vol_med_mus, val_ndsc_med_mus,
                    val_dsc_inf_mus, val_hau_avg_inf_mus, val_vol_inf_mus, val_ndsc_inf_mus,
                    val_dsc_sup_mus, val_hau_avg_sup_mus, val_vol_sup_mus, val_ndsc_sup_mus,
])

vals = vals.T

with open('/mnt/sda1/Repos/a-eye/Output/similarity/sim_reg_vs_nnunet_raw.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow(metrics)
    writer.writerows(vals)

Read values from csv

In [ ]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.pyplot import Line2D
import numpy as np

csv_file = pd.read_csv('/mnt/sda1/Repos/a-eye/Output/similarity/sim_reg_vs_nnunet_raw.csv')
# csv_file = csv_file[~csv_file['Subject'].isin([int(x) for x in bad_files])].reset_index(drop=True)

# Dataframes {DSC, nDSC, Volume (voxels)} separate labels for N=5 only
data_dsc = [csv_file['DSC_all'], csv_file['DSC_lens'], csv_file['DSC_globe'], csv_file['DSC_nerve'], csv_file['DSC_int_fat'], csv_file['DSC_ext_fat'], csv_file['DSC_lat_mus'], csv_file['DSC_med_mus'], csv_file['DSC_inf_mus'], csv_file['DSC_sup_mus']]
data_ndsc = [csv_file['nDSC_all'],  csv_file['nDSC_lens'], csv_file['nDSC_globe'], csv_file['nDSC_nerve'], csv_file['nDSC_int_fat'], csv_file['nDSC_ext_fat'], csv_file['nDSC_lat_mus'], csv_file['nDSC_med_mus'], csv_file['nDSC_inf_mus'], csv_file['nDSC_sup_mus']]
data_vol = [csv_file['Volume_all'], csv_file['Volume_lens'], csv_file['Volume_globe'], csv_file['Volume_nerve'], csv_file['Volume_int_fat'], csv_file['Volume_ext_fat'], csv_file['Volume_lat_mus'], csv_file['Volume_med_mus'], csv_file['Volume_inf_mus'], csv_file['Volume_sup_mus']]
data_haus = [csv_file['Haus_avg_all'], csv_file['Haus_avg_lens'], csv_file['Haus_avg_globe'], csv_file['Haus_avg_nerve'], csv_file['Haus_avg_int_fat'], csv_file['Haus_avg_ext_fat'], csv_file['Haus_avg_lat_mus'], csv_file['Haus_avg_med_mus'], csv_file['Haus_avg_inf_mus'], csv_file['Haus_avg_sup_mus']]

labels = ['lens', 'globe', 'nerve', 'intraconal fat', 'extraconal fat', 'lateral rectus muscle', 'medial rectus muscle', 'inferior rectus muscle', 'superior rectus muscle']
median = [np.around(np.median(x), 2) for x in data_dsc]
for i in range(len(labels)):
    print(labels[i], median[i])

In [ ]:
# Figure
fig, axs = plt.subplots(3, figsize=(20,10), sharex=True)
fig.canvas.manager.set_window_title('Similarity metrics ATLAS vs DL')
fig.suptitle('Similarity metrics ATLAS vs nnUNet')
fig.patch.set_facecolor('white')
# fig.tight_layout(pad=2)

# Set labels and titles
axs[0].set_title('DSC')
axs[0].set_ylim([0, 1])
axs[1].set_title('Hausdorff distance')
axs[1].set_ylim([0, 3])
axs[2].set_title('Volume difference')
axs[2].set_ylim([-2, 2])

# Outliers
flierprops = dict(markerfacecolor='0.9', markersize=2, linestyle='none')

# Boxplot & Swarmplot (points)
graph1 = sns.boxplot(data=data_dsc, ax=axs[0], flierprops=flierprops).set(ylabel="Value")
graph2 = sns.boxplot(data=data_haus, ax=axs[1], flierprops=flierprops).set(ylabel="Value")
graph3 = sns.boxplot(data=data_vol, ax=axs[2], flierprops=flierprops).set(ylabel="Value")
axs[2].set_xticklabels(['all','lens','globe','nerve','int_fat','ext_fat','lat_mus','med_mus','inf_mus','sup_mus'])

plt.show

# Save figure
plt.savefig('/mnt/sda1/Repos/a-eye/Output/similarity/sim_reg_vs_nnunet.png', bbox_inches='tight')

#### Outliers

In [ ]:
# remove nDSC columns from csv_file
csv_file = csv_file.drop(columns=['nDSC_all', 'nDSC_lens', 'nDSC_globe', 'nDSC_nerve', 'nDSC_int_fat', 'nDSC_ext_fat', 'nDSC_lat_mus', 'nDSC_med_mus', 'nDSC_inf_mus', 'nDSC_sup_mus'])
# reorder columns, first the DSCs, then the Hausdorff distances, then the volumes
csv_file = csv_file[['Subject', 'DSC_all', 'DSC_lens', 'DSC_globe', 'DSC_nerve', 'DSC_int_fat', 'DSC_ext_fat', 'DSC_lat_mus', 'DSC_med_mus', 'DSC_inf_mus', 'DSC_sup_mus', 'Haus_avg_all', 'Haus_avg_lens', 'Haus_avg_globe', 'Haus_avg_nerve', 'Haus_avg_int_fat', 'Haus_avg_ext_fat', 'Haus_avg_lat_mus', 'Haus_avg_med_mus', 'Haus_avg_inf_mus', 'Haus_avg_sup_mus', 'Volume_all', 'Volume_lens', 'Volume_globe', 'Volume_nerve', 'Volume_int_fat', 'Volume_ext_fat', 'Volume_lat_mus', 'Volume_med_mus', 'Volume_inf_mus', 'Volume_sup_mus']]

Normality test

In [ ]:
# Shapiro-Wilk test for normality for each column of csv_file
from scipy.stats import shapiro

for i in range(1, len(csv_file.columns)):
    stat, p = shapiro(csv_file.iloc[:,i])
    print(f'{csv_file.columns[i]}: Statistics={stat}, p={p}')
    alpha = 0.05
    if p > alpha:
        print('Sample looks Gaussian (fail to reject H0)')
    else:
        print('Sample does not look Gaussian (reject H0)')
    # histogram plot
    plt.hist(csv_file.iloc[:,i])
    plt.show()

Outliers - IQR

In [ ]:
outliers_df1 = pd.DataFrame(columns=['Column', 'Outlier Subject'])
bounds_df = pd.DataFrame(columns=['Column', 'Lower Bound', 'Upper Bound']) # dataframe for bounds

# Calculate the IQR for each column
Q1 = csv_file.quantile(0.25)
Q3 = csv_file.quantile(0.75)
IQR = Q3 - Q1

# Find the outliers for each column
for column in csv_file.columns[1:]:
    lower_bound = Q1[column] - 1.5 * IQR[column]
    upper_bound = Q3[column] + 1.5 * IQR[column]

    # Append bounds to the bounds_df DataFrame
    new_row = pd.DataFrame({'Column': [column], 'Lower Bound': [lower_bound], 'Upper Bound': [upper_bound]})
    bounds_df = pd.concat([bounds_df, new_row], ignore_index=True)

    column_outliers = csv_file[(csv_file[column] < lower_bound) | (csv_file[column] > upper_bound)]
    new_row = pd.DataFrame({'Column': [column], 'Outlier Subject': [column_outliers['Subject'].values]})
    outliers_df1 = pd.concat([outliers_df1, new_row], ignore_index=True)

In [ ]:
# Initialize a new DataFrame with the same structure as csv_file but filled with False
outliers_df2 = pd.DataFrame(False, index=csv_file.index, columns=csv_file.columns)

# Copy the 'Subject' column from csv_file to outliers_df
outliers_df2['Subject'] = csv_file['Subject']

# Calculate the IQR for each column excluding 'Subject'
Q1 = csv_file.drop('Subject', axis=1).quantile(0.25)
Q3 = csv_file.drop('Subject', axis=1).quantile(0.75)
IQR = Q3 - Q1

# Find the outliers for each column excluding 'Subject'
for column in csv_file.columns:
    if column != 'Subject':
        lower_bound = Q1[column] - 1.5 * IQR[column]
        upper_bound = Q3[column] + 1.5 * IQR[column]
        # Update outliers_df with True for outliers
        outliers_df2[column] = (csv_file[column] < lower_bound) | (csv_file[column] > upper_bound)

# Replace all False values with ''
outliers_df2 = outliers_df2.replace(False, '')

In [ ]:
# subdataframe of outliers_df2 with 'Subject' and DSC columns
outliers_df2_dsc = outliers_df2[['Subject', 'DSC_all', 'DSC_lens', 'DSC_globe', 'DSC_nerve', 'DSC_int_fat', 'DSC_ext_fat', 'DSC_lat_mus', 'DSC_med_mus', 'DSC_inf_mus', 'DSC_sup_mus']]
# subdataframe of outliers_df2 with 'Subject' and Hausdorff columns
outliers_df2_haus = outliers_df2[['Subject', 'Haus_avg_all', 'Haus_avg_lens', 'Haus_avg_globe', 'Haus_avg_nerve', 'Haus_avg_int_fat', 'Haus_avg_ext_fat', 'Haus_avg_lat_mus', 'Haus_avg_med_mus', 'Haus_avg_inf_mus', 'Haus_avg_sup_mus']]
# subdataframe of outliers_df2 with 'Subject' and Volume columns
outliers_df2_vol = outliers_df2[['Subject', 'Volume_all', 'Volume_lens', 'Volume_globe', 'Volume_nerve', 'Volume_int_fat', 'Volume_ext_fat', 'Volume_lat_mus', 'Volume_med_mus', 'Volume_inf_mus', 'Volume_sup_mus']]

In [ ]:
# make a list of 'subject' from outliers_df2_dsc that appears more than once as true
outliers_dsc = []
for column in outliers_df2_dsc.columns[1:]:
    outliers_dsc.append(outliers_df2_dsc[outliers_df2_dsc[column] == True]['Subject'].values)
outliers_dsc = [item for sublist in outliers_dsc for item in sublist]
outliers_dsc = [item for item in outliers_dsc if outliers_dsc.count(item) > 2]
# remove duplicates
outliers_dsc = list(dict.fromkeys(outliers_dsc))

In [ ]:
# make a list of 'subject' from outliers_df2_haus that appears more than once as true
outliers_haus = []
for column in outliers_df2_haus.columns[1:]:
    outliers_haus.append(outliers_df2_haus[outliers_df2_haus[column] == True]['Subject'].values)
outliers_haus = [item for sublist in outliers_haus for item in sublist]
outliers_haus = [item for item in outliers_haus if outliers_haus.count(item) > 2]
# remove duplicates
outliers_haus = list(dict.fromkeys(outliers_haus))

In [ ]:
# make a list of 'subject' from outliers_df2_vol that appears more than once as true
outliers_vol = []
for column in outliers_df2_vol.columns[1:]:
    outliers_vol.append(outliers_df2_vol[outliers_df2_vol[column] == True]['Subject'].values)
outliers_vol = [item for sublist in outliers_vol for item in sublist]
outliers_vol = [item for item in outliers_vol if outliers_vol.count(item) > 2]
# remove duplicates
outliers_vol = list(dict.fromkeys(outliers_vol))

In [ ]:
# joint dataframe of outliers_dsc, outliers_haus, outliers_vol without 'subject' duplicates
outliers_df = pd.DataFrame({'Subject': outliers_dsc + outliers_haus + outliers_vol})
# remove duplicates
outliers_df = outliers_df.drop_duplicates(subset=['Subject']).reset_index(drop=True)
# to list
outliers_list = outliers_df['Subject'].tolist()

Reports to look at

In [ ]:
import pandas as pd

qc2_atlas = pd.read_excel("/home/jaimebarranco/Desktop/MRI-QC/qc2_atlas.xlsx")
qc3_atlas = pd.read_excel("/home/jaimebarranco/Desktop/MRI-QC/qc3_vol_atlas.xlsx")
qc2_nnunet = pd.read_excel("/home/jaimebarranco/Desktop/MRI-QC/qc2_nnunet.xlsx")
qc3_nnunet = pd.read_excel("/home/jaimebarranco/Desktop/MRI-QC/qc3_vol_nnunet.xlsx")

In [ ]:
import pandas as pd

df_aux = pd.read_csv("/home/jaimebarranco/Desktop/MRI-QC/df_aux.csv")

# subdataframe of df_aux with the outliers_list
outliers_reports_df = df_aux[df_aux['subject'].isin(outliers_list)]
# add 'my_rate' column to outliers_reports_df
outliers_reports_df.loc[:, 'my_rate'] = 0.0
# order outliers_reports_df by 'report'
outliers_reports_df = outliers_reports_df.sort_values(by=['report'])
# remove subjects from outliers_reports_df that are in qc3_atlas - useful to view the non-repeated reports
outliers_reports_df = outliers_reports_df[~outliers_reports_df['subject'].isin(qc3_atlas['subject'])]
outliers_reports_df = outliers_reports_df[~outliers_reports_df['subject'].isin(qc2_atlas['subject'])]
outliers_reports_df = outliers_reports_df[~outliers_reports_df['subject'].isin(qc3_nnunet['subject'])]
outliers_reports_df = outliers_reports_df[~outliers_reports_df['subject'].isin(qc2_nnunet['subject'])]
# add column 'comments' to outliers_reports_df
outliers_reports_df.loc[:, 'comments'] = ''
# save outliers_reports_df to excel file
# outliers_reports_df.to_excel("/home/jaimebarranco/Desktop/MRI-QC/qc2_atlas.xlsx", index=False)

In [ ]:
# copy reports that appear in outliers_reports from /home/jaimebarranco/Desktop/MRI-QC/fetal/fetalqc_non-labeled-dataset to /home/jaimebarranco/Downloads/reports 
import shutil
import os

reports_folder = '/home/jaimebarranco/Downloads/reports/'
# create reports folder if it doesn't exist
if not os.path.exists(reports_folder):
    os.makedirs(reports_folder)
else:
    # remove content from reports folder
    for filename in os.listdir(reports_folder):
        file_path = os.path.join(reports_folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')

for report in outliers_reports_df['report']:
    src = f'/home/jaimebarranco/Desktop/MRI-QC/fetal/fetalqc_non-labeled-dataset/{report}_report.html'
    dst = f'/home/jaimebarranco/Downloads/reports/{report}_report.html'
    shutil.copy(src, dst)

In [ ]:
# replace row in outliers_reports_df with the same subject as in qc2_atlas
for i in range(len(qc2_atlas)):
    outliers_reports_df.loc[outliers_reports_df['subject'] == qc2_atlas.iloc[i]['subject'], 'my_rate'] = qc2_atlas.iloc[i]['my_rate']
    outliers_reports_df.loc[outliers_reports_df['subject'] == qc2_atlas.iloc[i]['subject'], 'comments'] = qc2_atlas.iloc[i]['comments']
    # replace row in outliers_reports_df with the same subject as in qc2_atlas

# replace row in outliers_reports_df with the same subject as in qc3_atlas
for i in range(len(qc3_atlas)):
    outliers_reports_df.loc[outliers_reports_df['subject'] == qc3_atlas.iloc[i]['subject'], 'my_rate'] = qc3_atlas.iloc[i]['my_rate']
    outliers_reports_df.loc[outliers_reports_df['subject'] == qc3_atlas.iloc[i]['subject'], 'comments'] = qc3_atlas.iloc[i]['comments']

# replace row in outliers_reports_df with the same subject as in qc2_nnunet
for i in range(len(qc2_nnunet)):
    outliers_reports_df.loc[outliers_reports_df['subject'] == qc2_nnunet.iloc[i]['subject'], 'my_rate'] = qc2_nnunet.iloc[i]['my_rate']
    outliers_reports_df.loc[outliers_reports_df['subject'] == qc2_nnunet.iloc[i]['subject'], 'comments'] = qc2_nnunet.iloc[i]['comments']

# replace row in outliers_reports_df with the same subject as in qc3_nnunet
for i in range(len(qc3_nnunet)):
    outliers_reports_df.loc[outliers_reports_df['subject'] == qc3_nnunet.iloc[i]['subject'], 'my_rate'] = qc3_nnunet.iloc[i]['my_rate']
    outliers_reports_df.loc[outliers_reports_df['subject'] == qc3_nnunet.iloc[i]['subject'], 'comments'] = qc3_nnunet.iloc[i]['comments']

In [ ]:
# save outliers_reports_df to excel file
outliers_reports_df.to_excel("/home/jaimebarranco/Desktop/MRI-QC/qc2_atlas_nnunet.xlsx", index=False)

In [ ]:
import pandas as pd 

qc2_atlas_nnunet = pd.read_excel("/home/jaimebarranco/Desktop/MRI-QC/qc2_atlas_nnunet.xlsx")

qc2_atlas_nnunet.to_csv("/home/jaimebarranco/Desktop/MRI-QC/qc2_atlas_nnunet.csv", index=False)

## Labeled dataset

### ATLAS vs nnUNet

Read csv

In [ ]:
from matplotlib.pyplot import Line2D
import pandas as pd

nnunet_path = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet/nnUNet_inference/nnUNet_inference_labeled_dataset/' 

# CSVs
csv_dsc_reg = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/3D_multilabel/experiment_0/test_orig/data_dsc_reg.csv'
csv_dsc_dl = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/3D_multilabel/experiment_0/test_orig/data_dsc.csv'
csv_dsc_nn = f'{nnunet_path}data_dsc_nn.csv'

# Pandas read CSV
df_reg = pd.read_csv(csv_dsc_reg)
df_dl = pd.read_csv(csv_dsc_dl)
df_nn = pd.read_csv(csv_dsc_nn)

# Gather data from generated dataframes
df_dsc = [
    df_reg['DSC_all'], df_nn['DSC_all'], 
    df_reg['DSC_lens'], df_nn['DSC_lens'], 
    df_reg['DSC_globe'], df_nn['DSC_globe'], 
    df_reg['DSC_nerve'], df_nn['DSC_nerve'], 
    df_reg['DSC_int_fat'], df_nn['DSC_int_fat'], 
    df_reg['DSC_ext_fat'], df_nn['DSC_ext_fat'], 
    df_reg['DSC_lat_mus'], df_nn['DSC_lat_mus'], 
    df_reg['DSC_med_mus'], df_nn['DSC_med_mus'], 
    df_reg['DSC_inf_mus'], df_nn['DSC_inf_mus'], 
    df_reg['DSC_sup_mus'], df_nn['DSC_sup_mus']]

Plot

In [ ]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.pyplot import Line2D
import numpy as np

# Figure
fig, axs = plt.subplots(figsize=(28,10))
fig.canvas.manager.set_window_title('DSC per label')
fig.patch.set_facecolor('white')

# Legend
legend_elements = [Line2D([0], [0], color='green', lw=4, label='Atlas-based'),
    Line2D([0], [0], color='purple', lw=4, label='nnUNet'),
    Line2D([], [], color='black', label='mean', marker='+', markersize=5, linestyle='None')]

# Axis
colours_palette = ['green','purple','green','purple','green','purple','green','purple','green','purple','green','purple','green','purple','green','purple','green','purple','green','purple']
meanpointprops = {"marker":"+", "markerfacecolor":"black", "markeredgecolor":"black", "markersize":"10"}
ax = sns.boxplot(data=df_dsc, palette=colours_palette, showmeans=True, meanprops=meanpointprops)
# ax = sns.swarmplot(data=df_dsc)
ax.set_title('DSC')
ax.legend(handles=legend_elements)
ax.set(ylabel="Value")
ax.set_xticklabels([
    f'all \n{np.around(np.mean(df_dsc[0]), 3)}', f'all \n{np.around(np.mean(df_dsc[1]), 3)}',
    f'lens \n{np.around(np.mean(df_dsc[2]), 3)}', f'lens \n{np.around(np.mean(df_dsc[3]), 3)}',
    f'globe \n{np.around(np.mean(df_dsc[4]), 3)}', f'globe \n{np.around(np.mean(df_dsc[5]), 3)}',
    f'nerve \n{np.around(np.mean(df_dsc[6]), 3)}', f'nerve \n{np.around(np.mean(df_dsc[7]), 3)}',
    f'int_fat \n{np.around(np.mean(df_dsc[8]), 3)}', f'int_fat \n{np.around(np.mean(df_dsc[9]), 3)}',
    f'ext_fat \n{np.around(np.mean(df_dsc[10]), 3)}', f'ext_fat \n{np.around(np.mean(df_dsc[11]), 3)}',
    f'lat_mus \n{np.around(np.mean(df_dsc[12]), 3)}', f'lat_mus \n{np.around(np.mean(df_dsc[12]), 3)}',
    f'med_mus \n{np.around(np.mean(df_dsc[14]), 3)}', f'med_mus \n{np.around(np.mean(df_dsc[13]), 3)}',
    f'inf_mus \n{np.around(np.mean(df_dsc[16]), 3)}', f'inf_mus \n{np.around(np.mean(df_dsc[14]), 3)}',
    f'sup_mus \n{np.around(np.mean(df_dsc[18]), 3)}', f'sup_mus \n{np.around(np.mean(df_dsc[15]), 3)}'])

ax.set_yticks(np.arange(0.5, 1.02, 0.02))

plt.show

# plt.savefig(f'{nnunet_path}reg_vs_nn.png', bbox_inches='tight')

### DL vs nnUNet

Read csv

In [ ]:
from matplotlib.pyplot import Line2D
import pandas as pd

nnunet_path = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet/nnUNet_inference/nnUNet_inference_labeled_dataset/' 

# CSVs
csv_dsc_reg = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/3D_multilabel/experiment_0/test_orig/data_dsc_reg.csv'
csv_dsc_dl = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/3D_multilabel/experiment_0/test_orig/data_dsc.csv'
csv_dsc_nn = f'{nnunet_path}data_dsc_nn.csv'

# Pandas read CSV
df_reg = pd.read_csv(csv_dsc_reg)
df_dl = pd.read_csv(csv_dsc_dl)
df_nn = pd.read_csv(csv_dsc_nn)

# Gather data from generated dataframes
df_dsc = [
    df_dl['DSC_all'], df_nn['DSC_all'], 
    df_dl['DSC_lens'], df_nn['DSC_lens'], 
    df_dl['DSC_globe'], df_nn['DSC_globe'], 
    df_dl['DSC_nerve'], df_nn['DSC_nerve'], 
    df_dl['DSC_int_fat'], df_nn['DSC_int_fat'], 
    df_dl['DSC_ext_fat'], df_nn['DSC_ext_fat'], 
    df_dl['DSC_lat_mus'], df_nn['DSC_lat_mus'], 
    df_dl['DSC_med_mus'], df_nn['DSC_med_mus'], 
    df_dl['DSC_inf_mus'], df_nn['DSC_inf_mus'], 
    df_dl['DSC_sup_mus'], df_nn['DSC_sup_mus']]

Plot

In [ ]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.pyplot import Line2D
import numpy as np

# Figure
fig, axs = plt.subplots(figsize=(28,10))
fig.canvas.manager.set_window_title('DSC per label')
fig.patch.set_facecolor('white')

# Legend
legend_elements = [Line2D([0], [0], color='orange', lw=4, label='DL'),
    Line2D([0], [0], color='purple', lw=4, label='nnUNet'),
    Line2D([], [], color='black', label='mean', marker='+', markersize=5, linestyle='None')]

# Axis
colours_palette = ['orange','purple','orange','purple','orange','purple','orange','purple','orange','purple','orange','purple','orange','purple','orange','purple','orange','purple','orange','purple']
meanpointprops = {"marker":"+", "markerfacecolor":"black", "markeredgecolor":"black", "markersize":"10"}
ax = sns.boxplot(data=df_dsc, palette=colours_palette, showmeans=True, meanprops=meanpointprops)
# ax = sns.swarmplot(data=df_dsc)
ax.set_title('DSC')
ax.legend(handles=legend_elements)
ax.set(ylabel="Value")
ax.set_xticklabels([
    f'all \n{np.around(np.mean(df_dsc[0]), 3)}', f'all \n{np.around(np.mean(df_dsc[1]), 3)}',
    f'lens \n{np.around(np.mean(df_dsc[2]), 3)}', f'lens \n{np.around(np.mean(df_dsc[3]), 3)}',
    f'globe \n{np.around(np.mean(df_dsc[4]), 3)}', f'globe \n{np.around(np.mean(df_dsc[5]), 3)}',
    f'nerve \n{np.around(np.mean(df_dsc[6]), 3)}', f'nerve \n{np.around(np.mean(df_dsc[7]), 3)}',
    f'int_fat \n{np.around(np.mean(df_dsc[8]), 3)}', f'int_fat \n{np.around(np.mean(df_dsc[9]), 3)}',
    f'ext_fat \n{np.around(np.mean(df_dsc[10]), 3)}', f'ext_fat \n{np.around(np.mean(df_dsc[11]), 3)}',
    f'lat_mus \n{np.around(np.mean(df_dsc[12]), 3)}', f'lat_mus \n{np.around(np.mean(df_dsc[12]), 3)}',
    f'med_mus \n{np.around(np.mean(df_dsc[14]), 3)}', f'med_mus \n{np.around(np.mean(df_dsc[13]), 3)}',
    f'inf_mus \n{np.around(np.mean(df_dsc[16]), 3)}', f'inf_mus \n{np.around(np.mean(df_dsc[14]), 3)}',
    f'sup_mus \n{np.around(np.mean(df_dsc[18]), 3)}', f'sup_mus \n{np.around(np.mean(df_dsc[15]), 3)}'])
ax.set_yticks(np.arange(0.5, 1.02, 0.02))

plt.show

plt.savefig(f'{nnunet_path}dl_vs_nn.png', bbox_inches='tight')

### ATLAS vs DL vs nnUNet

Read csv

In [ ]:
from matplotlib.pyplot import Line2D
import pandas as pd

nnunet_path = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/nnUNet/nnUNet/nnUNet_inference/nnUNet_inference_labeled_dataset/' 

# CSVs
csv_dsc_reg = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/3D_multilabel/experiment_0/test_orig/data_dsc_reg.csv'
csv_dsc_dl = '/mnt/sda1/Repos/a-eye/a-eye_segmentation/deep_learning/3D_multilabel/experiment_0/test_orig/data_dsc.csv'
csv_dsc_nn = f'{nnunet_path}data_dsc_nn.csv'

# Pandas read CSV
df_reg = pd.read_csv(csv_dsc_reg)
df_dl = pd.read_csv(csv_dsc_dl)
df_nn = pd.read_csv(csv_dsc_nn)

# Gather data from generated dataframes
df_dsc = [
    df_reg['DSC_all'], df_dl['DSC_all'], df_nn['DSC_all'],
    df_reg['DSC_lens'], df_dl['DSC_lens'], df_nn['DSC_lens'],
    df_reg['DSC_globe'], df_dl['DSC_globe'], df_nn['DSC_globe'],
    df_reg['DSC_nerve'], df_dl['DSC_nerve'], df_nn['DSC_nerve'],
    df_reg['DSC_int_fat'], df_dl['DSC_int_fat'], df_nn['DSC_int_fat'],
    df_reg['DSC_ext_fat'], df_dl['DSC_ext_fat'], df_nn['DSC_ext_fat'],
    df_reg['DSC_lat_mus'], df_dl['DSC_lat_mus'], df_nn['DSC_lat_mus'],
    df_reg['DSC_med_mus'], df_dl['DSC_med_mus'], df_nn['DSC_med_mus'],
    df_reg['DSC_inf_mus'], df_dl['DSC_inf_mus'], df_nn['DSC_inf_mus'],
    df_reg['DSC_sup_mus'], df_dl['DSC_sup_mus'], df_nn['DSC_sup_mus']]


Plot

In [ ]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.pyplot import Line2D
import numpy as np

# Figure
fig, axs = plt.subplots(figsize=(30,10))
fig.canvas.manager.set_window_title('DSC per label')
fig.patch.set_facecolor('white')

# Legend
legend_elements = [
    Line2D([0], [0], color='green', lw=4, label='Atlas-based'),
    Line2D([0], [0], color='orange', lw=4, label='DL'),
    Line2D([0], [0], color='purple', lw=4, label='nnUNet'),
    Line2D([], [], color='black', label='mean', marker='+', markersize=5, linestyle='None')]

# Axis
colours_palette = ['green','orange','purple']*10
meanpointprops = {"marker":"+", "markerfacecolor":"black", "markeredgecolor":"black", "markersize":"10"}
ax = sns.boxplot(data=df_dsc, palette=colours_palette, showmeans=True, meanprops=meanpointprops)
# ax = sns.swarmplot(data=df_dsc)
ax.set_title('DSC')
ax.legend(handles=legend_elements)
ax.set(ylabel="Value")
ax.set_xticklabels([
    f'all \n{np.around(np.mean(df_dsc[0]), 3)}', f'all \n{np.around(np.mean(df_dsc[1]), 3)}', f'all \n{np.around(np.mean(df_dsc[2]), 3)}',
    f'lens \n{np.around(np.mean(df_dsc[3]), 3)}', f'lens \n{np.around(np.mean(df_dsc[4]), 3)}', f'lens \n{np.around(np.mean(df_dsc[5]), 3)}',
    f'globe \n{np.around(np.mean(df_dsc[6]), 3)}', f'globe \n{np.around(np.mean(df_dsc[7]), 3)}', f'globe \n{np.around(np.mean(df_dsc[8]), 3)}',
    f'nerve \n{np.around(np.mean(df_dsc[9]), 3)}', f'nerve \n{np.around(np.mean(df_dsc[10]), 3)}', f'nerve \n{np.around(np.mean(df_dsc[11]), 3)}',
    f'int_fat \n{np.around(np.mean(df_dsc[12]), 3)}', f'int_fat \n{np.around(np.mean(df_dsc[13]), 3)}', f'int_fat \n{np.around(np.mean(df_dsc[14]), 3)}',
    f'ext_fat \n{np.around(np.mean(df_dsc[15]), 3)}', f'ext_fat \n{np.around(np.mean(df_dsc[16]), 3)}', f'ext_fat \n{np.around(np.mean(df_dsc[17]), 3)}',
    f'lat_mus \n{np.around(np.mean(df_dsc[18]), 3)}', f'lat_mus \n{np.around(np.mean(df_dsc[19]), 3)}', f'lat_mus \n{np.around(np.mean(df_dsc[20]), 3)}',
    f'med_mus \n{np.around(np.mean(df_dsc[21]), 3)}', f'med_mus \n{np.around(np.mean(df_dsc[22]), 3)}', f'med_mus \n{np.around(np.mean(df_dsc[23]), 3)}',
    f'inf_mus \n{np.around(np.mean(df_dsc[24]), 3)}', f'inf_mus \n{np.around(np.mean(df_dsc[25]), 3)}', f'inf_mus \n{np.around(np.mean(df_dsc[26]), 3)}',
    f'sup_mus \n{np.around(np.mean(df_dsc[27]), 3)}', f'sup_mus \n{np.around(np.mean(df_dsc[28]), 3)}', f'sup_mus \n{np.around(np.mean(df_dsc[29]), 3)}'])
ax.set_yticks(np.arange(0.5, 1.02, 0.02))

plt.show

plt.savefig(f'{nnunet_path}reg_vs_dl_vs_nn.png', bbox_inches='tight')

## New manual annotations

QC1: remove subjects from QC1

In [ ]:
import pandas as pd

df_aux = pd.read_csv("/home/jaimebarranco/Desktop/MRI-QC/df_aux.csv")
qc1 = pd.read_csv("/home/jaimebarranco/Desktop/MRI-QC/qc1.csv")

In [ ]:
list_qc1 = df_aux.loc[df_aux['bids'].isin(qc1['bids'])]['subject'].tolist()
list_qc1 = [str(x) for x in list_qc1]

### ATLAS vs Manual

Cropping

In [ ]:
import numpy as np
import os,sys, glob
import nibabel as nib
import SimpleITK as sitk
import pandas as pd

# Paths
atlas_seg_path = '/home/jaimebarranco/Desktop/new_manual_annotations/segmentations/atlas'
atlas_cropped_path = '/home/jaimebarranco/Desktop/new_manual_annotations/segmentations/atlas_cropped'
manual_seg_path = '/home/jaimebarranco/Desktop/new_manual_annotations/segmentations/manual'
output_path = '/home/jaimebarranco/Desktop/new_manual_annotations/segmentations/manual_cropped'
if not os.path.exists(output_path):
    os.makedirs(output_path)

bound = 15 # for the expanded bounding box

i=0
for file in sorted(glob.glob(atlas_seg_path+'/*.nii.gz')):

    subject = os.path.basename(file).split('.')[0]
    # print(f'subject = {subject}')

    if os.path.exists(os.path.join(output_path, f'{subject}.nii.gz')):
        continue
    else:
        # Paths
        atlas_seg = f'{atlas_seg_path}/{subject}.nii.gz'
        manual_seg = f'{manual_seg_path}/{subject}.nii.gz'
        atlas_cropped = f'{atlas_cropped_path}/{subject}.nii.gz'
        
        # sizes
        atlas_image = sitk.ReadImage(atlas_seg)
        atlas_cropped_image = sitk.ReadImage(atlas_cropped)
        manual_image = sitk.ReadImage(manual_seg)
        atlas_x_size, atlas_y_size, atlas_z_size = atlas_image.GetSize()
        atlas_cropped_x_size, atlas_cropped_y_size, atlas_cropped_z_size = atlas_cropped_image.GetSize()
        manual_x_size, manual_y_size, manual_z_size = manual_image.GetSize()
        print(f"atlas size: ({atlas_x_size}, {atlas_y_size}, {atlas_z_size})")
        print(f"atlas cropped size: ({atlas_cropped_x_size}, {atlas_cropped_y_size}, {atlas_cropped_z_size})")
        print(f"manual size: ({manual_x_size}, {manual_y_size}, {manual_z_size})")

        # Bounding box
        all_segments_mask = atlas_image > 0
        lsif = sitk.LabelStatisticsImageFilter() # It requires intensity and label images
        lsif.Execute(atlas_image, all_segments_mask) # Mask! Where all the labels are 1!
        bounding_box = np.array(lsif.GetBoundingBox(1)) # GetBoundingBox(label)
        print(f"Bounding box:  {bounding_box}") # [xmin, xmax, ymin, ymax, zmin, zmax]
        print(f'New manual size: ({bounding_box[1]-bounding_box[0]}, {bounding_box[3]-bounding_box[2]}, {bounding_box[5]-bounding_box[4]})')

        # Expand bounding box
        bounding_box_expanded = bounding_box.copy()
        bounding_box_expanded[0::2] -= bound # even indexes
        bounding_box_expanded[1::2] += bound # odd indexes
        print(f"Expanded bounding box: {bounding_box_expanded}")
        print(f'new manual size expanded: ({bounding_box_expanded[1]-bounding_box_expanded[0]}, {bounding_box_expanded[3]-bounding_box_expanded[2]}, {bounding_box_expanded[5]-bounding_box_expanded[4]})')

        # Limits
        if bounding_box_expanded[0] < 0: bounding_box_expanded[0] = 0
        if bounding_box_expanded[1] > manual_x_size: bounding_box_expanded[1] = manual_x_size
        if bounding_box_expanded[2] < 0: bounding_box_expanded[2] = 0
        if bounding_box_expanded[3] > manual_y_size: bounding_box_expanded[3] = manual_y_size
        if bounding_box_expanded[4] < 0: bounding_box_expanded[4] = 0
        if bounding_box_expanded[5] > manual_z_size: bounding_box_expanded[5] = manual_z_size
        print(f"Expanded bounding box after limits: {bounding_box_expanded}")
        print(f'new manual size expanded after limits: ({bounding_box_expanded[1]-bounding_box_expanded[0]}, {bounding_box_expanded[3]-bounding_box_expanded[2]}, {bounding_box_expanded[5]-bounding_box_expanded[4]})')

        # Crop
        manual_crop = manual_image[int(bounding_box_expanded[0]):int(bounding_box_expanded[1]), # x
                            int(bounding_box_expanded[2]):int(bounding_box_expanded[3]), # y
                            int(bounding_box_expanded[4]):int(bounding_box_expanded[5])] # z
        sitk.WriteImage(manual_crop, f'{output_path}/{subject}.nii.gz')

        i+=1
        # if (i==4):
        #     break             

Compute sim. metrics

In [ ]:
import SimpleITK as sitk
import numpy as np
import pandas as pd
import os
import nibabel as nb

# nDSC (normalized DSC)
def dice_norm_metric(ground_truth, predictions):
    '''
    For a single example returns DSC_norm, fpr, fnr
    '''

    # Reference for normalized DSC
    r = 0.001 # It should be 1/N*(np.sum(voxels_label[i])/np.sum(voxels_image[i])) i belonging to training set
    # Cast to float32 type
    gt = ground_truth.astype("float32")
    seg = predictions.astype("float32")
    im_sum = np.sum(seg) + np.sum(gt)
    if im_sum == 0:
        return 1.0, 1.0, 1.0
    else:
        if np.sum(gt) == 0:
            k = 1.0
        else:
            k = (1 - r) * np.sum(gt) / (r * (len(gt.flatten()) - np.sum(gt)))
        tp = np.sum(seg[gt == 1])
        fp = np.sum(seg[gt == 0])
        fn = np.sum(gt[seg == 0])
        fp_scaled = k * fp
        dsc_norm = 2 * tp / (fp_scaled + 2 * tp + fn)

        fpr = fp / (len(gt.flatten()) - np.sum(gt))
        if np.sum(gt) == 0:
            fnr = 1.0
        else:
            fnr = fn / np.sum(gt)
        return dsc_norm # fpr, fnr

# Paths - segmentation results
reg_dir = '/home/jaimebarranco/Desktop/new_manual_annotations/segmentations/atlas_cropped'
manual_dir = '/home/jaimebarranco/Desktop/new_manual_annotations/segmentations/manual_cropped'

list_subjects = sorted(os.listdir(reg_dir))
list_subjects_clean = []

for file in list_subjects:
    # Name subject
    name_subject = file.split('.')[0]
    # only non-excluded subjects
    if name_subject not in list_qc1:
        print(f'subject: {name_subject}')
        list_subjects_clean.append(name_subject)
        continue

# List of volumes for each individual label and subject
len_path = len(list_subjects_clean) # elements in gt path = number of subjects
name_subject = [None]*len_path

# Save values in an array
# All labels
val_dsc             = np.zeros(len_path)
# val_hau             = np.zeros(len(rest_subjects))
val_hau_avg         = np.zeros(len_path)
val_vol             = np.zeros(len_path)
val_ndsc            = np.zeros(len_path)
# Lens
val_dsc_lens        = np.zeros(len_path)
val_hau_avg_lens    = np.zeros(len_path)
val_vol_lens        = np.zeros(len_path)
val_ndsc_lens       = np.zeros(len_path)
# Globe
val_dsc_globe       = np.zeros(len_path)
val_hau_avg_globe   = np.zeros(len_path)
val_vol_globe       = np.zeros(len_path)
val_ndsc_globe      = np.zeros(len_path)
# Optic nerve  
val_dsc_nerve       = np.zeros(len_path)
val_hau_avg_nerve   = np.zeros(len_path)
val_vol_nerve       = np.zeros(len_path)
val_ndsc_nerve      = np.zeros(len_path)
# Intraconal fat
val_dsc_int_fat     = np.zeros(len_path)
val_hau_avg_int_fat = np.zeros(len_path)
val_vol_int_fat     = np.zeros(len_path)
val_ndsc_int_fat    = np.zeros(len_path)
# Extraconal fat
val_dsc_ext_fat     = np.zeros(len_path)
val_hau_avg_ext_fat = np.zeros(len_path)
val_vol_ext_fat     = np.zeros(len_path)
val_ndsc_ext_fat    = np.zeros(len_path)
# Lateral rectus muscle
val_dsc_lat_mus     = np.zeros(len_path)
val_hau_avg_lat_mus = np.zeros(len_path)
val_vol_lat_mus     = np.zeros(len_path)
val_ndsc_lat_mus    = np.zeros(len_path)
# Medial rectus muscle
val_dsc_med_mus     = np.zeros(len_path)
val_hau_avg_med_mus = np.zeros(len_path)
val_vol_med_mus     = np.zeros(len_path)
val_ndsc_med_mus    = np.zeros(len_path)
# Inferior rectus muscle
val_dsc_inf_mus     = np.zeros(len_path)
val_hau_avg_inf_mus = np.zeros(len_path)
val_vol_inf_mus     = np.zeros(len_path)
val_ndsc_inf_mus    = np.zeros(len_path)
# Superior rectus muscle
val_dsc_sup_mus     = np.zeros(len_path)
val_hau_avg_sup_mus = np.zeros(len_path)
val_vol_sup_mus     = np.zeros(len_path)
val_ndsc_sup_mus    = np.zeros(len_path)

reader = sitk.ImageFileReader()

i=0
for file in list_subjects_clean:

    # Name subject
    name_subject[i] = file

    # ATLAS-based results
    reg_lab_path = f'{reg_dir}/{name_subject[i]}.nii.gz'
    reader.SetFileName(reg_lab_path)
    reg_lab_sitk = sitk.Cast(reader.Execute(), sitk.sitkUInt8)
    reg_lab_arr = sitk.GetArrayFromImage(reg_lab_sitk)

    # MANUAL-based results
    dl_lab_path = f'{manual_dir}/{name_subject[i]}.nii.gz'
    # check if the file exists
    if not os.path.exists(dl_lab_path):
        print('file does not exist')
        i+=1
        continue
    reader.SetFileName(dl_lab_path)
    dl_lab_sitk = sitk.Cast(reader.Execute(), sitk.sitkUInt8)
    dl_lab_arr = sitk.GetArrayFromImage(dl_lab_sitk)
    
    # Image size
    im_lab_size = reg_lab_arr.shape[0]*reg_lab_arr.shape[1]*reg_lab_arr.shape[2]

    # LENS
    if np.count_nonzero(reg_lab_arr==1)==0 or np.count_nonzero(dl_lab_arr==1)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==1, dl_lab_sitk==1)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_lens[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_lens[i] = vol
        # Hausdorff distance
        hausdorf = sitk.HausdorffDistanceImageFilter()
        hausdorf.Execute(reg_lab_sitk==1, dl_lab_sitk==1)
        hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
        val_hau_avg_lens[i] = hausdorf_distance_avg
        # nDSC
        nDSC = dice_norm_metric(reg_lab_arr==1, dl_lab_arr==1)
        val_ndsc_lens[i] = nDSC
    
    # GLOBE EX LENS
    if np.count_nonzero(reg_lab_arr==2)==0 or np.count_nonzero(dl_lab_arr==2)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter 
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==2, dl_lab_sitk==2)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_globe[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_globe[i] = vol
        # Hausdorff distance
        hausdorf = sitk.HausdorffDistanceImageFilter()
        hausdorf.Execute(reg_lab_sitk==2, dl_lab_sitk==2)
        hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
        val_hau_avg_globe[i] = hausdorf_distance_avg
        # nDSC
        nDSC = dice_norm_metric(reg_lab_arr==2, dl_lab_arr==2)
        val_ndsc_globe[i] = nDSC

    # OPTIC NERVE
    if np.count_nonzero(reg_lab_arr==3)==0 or np.count_nonzero(dl_lab_arr==3)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter 
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==3, dl_lab_sitk==3)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_nerve[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_nerve[i] = vol
        # Hausdorff distance
        hausdorf = sitk.HausdorffDistanceImageFilter()
        hausdorf.Execute(reg_lab_sitk==3, dl_lab_sitk==3)
        hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
        val_hau_avg_nerve[i] = hausdorf_distance_avg
        # nDSC
        nDSC = dice_norm_metric(reg_lab_arr==3, dl_lab_arr==3)
        val_ndsc_nerve[i] = nDSC

    # INTRACONAL FAT
    if np.count_nonzero(reg_lab_arr==4)==0 or np.count_nonzero(dl_lab_arr==4)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter 
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==4, dl_lab_sitk==4)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_int_fat[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_int_fat[i] = vol
        # Hausdorff distance
        hausdorf = sitk.HausdorffDistanceImageFilter()
        hausdorf.Execute(reg_lab_sitk==4, dl_lab_sitk==4)
        hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
        val_hau_avg_int_fat[i] = hausdorf_distance_avg
        # nDSC
        nDSC = dice_norm_metric(reg_lab_arr==4, dl_lab_arr==4)
        val_ndsc_int_fat[i] = nDSC

    # EXTRACONAL FAT
    if np.count_nonzero(reg_lab_arr==5)==0 or np.count_nonzero(dl_lab_arr==5)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter 
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==5, dl_lab_sitk==5)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_ext_fat[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_ext_fat[i] = vol
        # Hausdorff distance
        hausdorf = sitk.HausdorffDistanceImageFilter()
        hausdorf.Execute(reg_lab_sitk==5, dl_lab_sitk==5)
        hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
        val_hau_avg_ext_fat[i] = hausdorf_distance_avg
        # nDSC
        nDSC = dice_norm_metric(reg_lab_arr==5, dl_lab_arr==5)
        val_ndsc_ext_fat[i] = nDSC

    # LATERAL RECTUS MUSCLE
    if np.count_nonzero(reg_lab_arr==6)==0 or np.count_nonzero(dl_lab_arr==6)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==6, dl_lab_sitk==6)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_lat_mus[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_lat_mus[i] = vol
        # Hausdorff distance
        hausdorf = sitk.HausdorffDistanceImageFilter()
        hausdorf.Execute(reg_lab_sitk==6, dl_lab_sitk==6)
        hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
        val_hau_avg_lat_mus[i] = hausdorf_distance_avg
        # nDSC
        nDSC = dice_norm_metric(reg_lab_arr==6, dl_lab_arr==6)
        val_ndsc_lat_mus[i] = nDSC

    # MEDIAL RECTUS MUSCLE
    if np.count_nonzero(reg_lab_arr==7)==0 or np.count_nonzero(dl_lab_arr==7)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter 
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==7, dl_lab_sitk==7)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_med_mus[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_med_mus[i] = vol
        # Hausdorff distance
        if np.count_nonzero(reg_lab_arr==7)==0 or np.count_nonzero(dl_lab_arr==7)==0: val_hau_avg_lens[i] = 100
        else :
            hausdorf = sitk.HausdorffDistanceImageFilter()
            hausdorf.Execute(reg_lab_sitk==7, dl_lab_sitk==7)
            hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
            val_hau_avg_med_mus[i] = hausdorf_distance_avg
        # nDSC
        nDSC = dice_norm_metric(reg_lab_arr==7, dl_lab_arr==7)
        val_ndsc_med_mus[i] = nDSC

    # INFERIOR RECTUS MUSCLE
    if np.count_nonzero(reg_lab_arr==8)==0 or np.count_nonzero(dl_lab_arr==8)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==8, dl_lab_sitk==8)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_inf_mus[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_inf_mus[i] = vol
        # Hausdorff distance
        if np.count_nonzero(reg_lab_arr==8)==0 or np.count_nonzero(dl_lab_arr==8)==0: val_hau_avg_lens[i] = 100
        else :
            hausdorf = sitk.HausdorffDistanceImageFilter()
            hausdorf.Execute(reg_lab_sitk==8, dl_lab_sitk==8)
            hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
            val_hau_avg_inf_mus[i] = hausdorf_distance_avg
        # nDSC
        nDSC = dice_norm_metric(reg_lab_arr==8, dl_lab_arr==8)
        val_ndsc_inf_mus[i] = nDSC

    # SUPERIOR RECTUS MUSCLE
    if np.count_nonzero(reg_lab_arr==9)==0 or np.count_nonzero(dl_lab_arr==9)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==9, dl_lab_sitk==9)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_sup_mus[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_sup_mus[i] = vol
        # Hausdorff distance
        hausdorf = sitk.HausdorffDistanceImageFilter()
        hausdorf.Execute(reg_lab_sitk==9, dl_lab_sitk==9)
        hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
        val_hau_avg_sup_mus[i] = hausdorf_distance_avg
        # nDSCvol
        nDSC = dice_norm_metric(reg_lab_arr==9, dl_lab_arr==9)
        val_ndsc_sup_mus[i] = nDSC

    # ALL LABELS
    # DSC
    dsc = (val_dsc_lens[i]+val_dsc_globe[i]+val_dsc_nerve[i]+val_dsc_int_fat[i]+val_dsc_ext_fat[i]+val_dsc_lat_mus[i]+val_dsc_med_mus[i]+val_dsc_inf_mus[i]+val_dsc_sup_mus[i])/9
    val_dsc[i] = dsc
    # Volume
    vol = (val_vol_lens[i]+val_vol_globe[i]+val_vol_nerve[i]+val_vol_int_fat[i]+val_vol_ext_fat[i]+val_vol_lat_mus[i]+val_vol_med_mus[i]+val_vol_inf_mus[i]+val_vol_sup_mus[i])/9
    val_vol[i] = vol
    # Hausdorff distance
    hau_avg = (val_hau_avg_lens[i]+val_hau_avg_globe[i]+val_hau_avg_nerve[i]+val_hau_avg_int_fat[i]+val_hau_avg_ext_fat[i]+val_hau_avg_lat_mus[i]+val_hau_avg_med_mus[i]+val_hau_avg_inf_mus[i]+val_hau_avg_sup_mus[i])/9
    val_hau_avg[i] = hau_avg
    # nDSC
    nDSC = (val_ndsc_lens[i]+val_ndsc_globe[i]+val_ndsc_nerve[i]+val_ndsc_int_fat[i]+val_ndsc_ext_fat[i]+val_ndsc_lat_mus[i]+val_ndsc_med_mus[i]+val_ndsc_inf_mus[i]+val_ndsc_sup_mus[i])/9
    val_ndsc[i] = nDSC

    i+=1
    # if i==1:
    #     break

Write values to csv

In [ ]:
import csv

metrics = [
    'Subject',  'DSC_all',     'Haus_avg_all',     'Volume_all',     'nDSC_all', 
                'DSC_lens',    'Haus_avg_lens',    'Volume_lens',    'nDSC_lens',  
                'DSC_globe',   'Haus_avg_globe',   'Volume_globe',   'nDSC_globe', 
                'DSC_nerve',   'Haus_avg_nerve',   'Volume_nerve',   'nDSC_nerve', 
                'DSC_int_fat', 'Haus_avg_int_fat', 'Volume_int_fat', 'nDSC_int_fat',
                'DSC_ext_fat', 'Haus_avg_ext_fat', 'Volume_ext_fat', 'nDSC_ext_fat',
                'DSC_lat_mus', 'Haus_avg_lat_mus', 'Volume_lat_mus', 'nDSC_lat_mus',
                'DSC_med_mus', 'Haus_avg_med_mus', 'Volume_med_mus', 'nDSC_med_mus',
                'DSC_inf_mus', 'Haus_avg_inf_mus', 'Volume_inf_mus', 'nDSC_inf_mus',
                'DSC_sup_mus', 'Haus_avg_sup_mus', 'Volume_sup_mus', 'nDSC_sup_mus'
]

vals = np.array([
    name_subject,   val_dsc,         val_hau_avg,         val_vol,         val_ndsc,        
                    val_dsc_lens,    val_hau_avg_lens,    val_vol_lens,    val_ndsc_lens,   
                    val_dsc_globe,   val_hau_avg_globe,   val_vol_globe,   val_ndsc_globe,  
                    val_dsc_nerve,   val_hau_avg_nerve,   val_vol_nerve,   val_ndsc_nerve,  
                    val_dsc_int_fat, val_hau_avg_int_fat, val_vol_int_fat, val_ndsc_int_fat,
                    val_dsc_ext_fat, val_hau_avg_ext_fat, val_vol_ext_fat, val_ndsc_ext_fat,
                    val_dsc_lat_mus, val_hau_avg_lat_mus, val_vol_lat_mus, val_ndsc_lat_mus,
                    val_dsc_med_mus, val_hau_avg_med_mus, val_vol_med_mus, val_ndsc_med_mus,
                    val_dsc_inf_mus, val_hau_avg_inf_mus, val_vol_inf_mus, val_ndsc_inf_mus,
                    val_dsc_sup_mus, val_hau_avg_sup_mus, val_vol_sup_mus, val_ndsc_sup_mus,
])

vals = vals.T

with open('/home/jaimebarranco/Desktop/new_manual_annotations/similarity/sim_manual_atlas_N43.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow(metrics)
    writer.writerows(vals)

Read csv

In [ ]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.pyplot import Line2D
import numpy as np
import plotly.express as px

csv_file = pd.read_csv('/home/jaimebarranco/Desktop/new_manual_annotations/similarity/sim_manual_atlas_N43.csv')

# Dataframes
data_dsc = [csv_file['DSC_all'], csv_file['DSC_lens'], csv_file['DSC_globe'], csv_file['DSC_nerve'], csv_file['DSC_int_fat'], csv_file['DSC_ext_fat'], csv_file['DSC_lat_mus'], csv_file['DSC_med_mus'], csv_file['DSC_inf_mus'], csv_file['DSC_sup_mus']]
data_ndsc = [csv_file['nDSC_all'],  csv_file['nDSC_lens'], csv_file['nDSC_globe'], csv_file['nDSC_nerve'], csv_file['nDSC_int_fat'], csv_file['nDSC_ext_fat'], csv_file['nDSC_lat_mus'], csv_file['nDSC_med_mus'], csv_file['nDSC_inf_mus'], csv_file['nDSC_sup_mus']]
data_vol = [csv_file['Volume_all'], csv_file['Volume_lens'], csv_file['Volume_globe'], csv_file['Volume_nerve'], csv_file['Volume_int_fat'], csv_file['Volume_ext_fat'], csv_file['Volume_lat_mus'], csv_file['Volume_med_mus'], csv_file['Volume_inf_mus'], csv_file['Volume_sup_mus']]
data_haus = [csv_file['Haus_avg_all'], csv_file['Haus_avg_lens'], csv_file['Haus_avg_globe'], csv_file['Haus_avg_nerve'], csv_file['Haus_avg_int_fat'], csv_file['Haus_avg_ext_fat'], csv_file['Haus_avg_lat_mus'], csv_file['Haus_avg_med_mus'], csv_file['Haus_avg_inf_mus'], csv_file['Haus_avg_sup_mus']]

labels = ['overall', 'lens', 'globe', 'nerve', 'intraconal fat', 'extraconal fat', 'lateral rectus muscle', 'medial rectus muscle', 'inferior rectus muscle', 'superior rectus muscle']
median = [np.around(np.median(x), 2) for x in data_dsc]
print('median DSC:')
for i in range(len(labels)):
    print(labels[i], median[i])

In [ ]:
# Figure
fig, axs = plt.subplots(3, figsize=(20,10), sharex=True)
fig.canvas.manager.set_window_title(f'Similarity metrics MANUAL vs ATLAS (N={len(csv_file)})')
fig.suptitle(f'Similarity metrics MANUAL vs ATLAS (N={len(csv_file)})')
fig.patch.set_facecolor('white')
# fig.tight_layout(pad=2)

# Set labels and titles
axs[0].set_title('DSC')
axs[0].set_ylim([0, 1])
axs[1].set_title('Hausdorff distance')
axs[1].set_ylim([0, 3])
axs[2].set_title('Volume difference')
axs[2].set_ylim([-2, 2])

# Outliers
flierprops = dict(markerfacecolor='0.9', markersize=2, linestyle='none')

# Boxplot & Swarmplot (points)
graph1 = sns.boxplot(data=data_dsc, ax=axs[0], flierprops=flierprops).set(ylabel="Value")
# graph1 = sns.swarmplot(data=data_dsc, ax=axs[0]).set(ylabel="Value")
graph2 = sns.boxplot(data=data_haus, ax=axs[1], flierprops=flierprops).set(ylabel="Value")
graph3 = sns.boxplot(data=data_vol, ax=axs[2], flierprops=flierprops).set(ylabel="Value")
axs[2].set_xticklabels(['all','lens','globe','nerve','int_fat','ext_fat','lat_mus','med_mus','inf_mus','sup_mus'])

plt.show

# Save figure
# plt.savefig('/home/jaimebarranco/Desktop/new_manual_annotations/similarity/sim_manual_atlas_N43.png', bbox_inches='tight')

In [ ]:
# check annotator bias for the lens' dsc
fig = px.scatter(y=data_dsc[1], labels={'x':'Subject', 'y':'Value'}, title='DSC Lens', height=600, width=800)
fig.update_traces(marker=dict(size=6, opacity=0.5))
fig.show()
# fig.write_image('/home/jaimebarranco/Desktop/new_manual_annotations/similarity/lens_dsc_atlas_manual.png')

# It seems Philip (first 20 subjects [4-23]) has more discrepancy with the atlas, but it also means that he has higher and lower values than
# Adrian. Adrian (second 14 subjects [23-36]), however, seems to be more focused on the 0.4-0.7 range.

#### Outliers

In [ ]:
# remove nDSC columns from csv_file
csv_file = csv_file.drop(columns=['nDSC_all', 'nDSC_lens', 'nDSC_globe', 'nDSC_nerve', 'nDSC_int_fat', 'nDSC_ext_fat', 'nDSC_lat_mus', 'nDSC_med_mus', 'nDSC_inf_mus', 'nDSC_sup_mus'])
# reorder columns, first the DSCs, then the Hausdorff distances, then the volumes
csv_file = csv_file[['Subject', 'DSC_all', 'DSC_lens', 'DSC_globe', 'DSC_nerve', 'DSC_int_fat', 'DSC_ext_fat', 'DSC_lat_mus', 'DSC_med_mus', 'DSC_inf_mus', 'DSC_sup_mus', 'Haus_avg_all', 'Haus_avg_lens', 'Haus_avg_globe', 'Haus_avg_nerve', 'Haus_avg_int_fat', 'Haus_avg_ext_fat', 'Haus_avg_lat_mus', 'Haus_avg_med_mus', 'Haus_avg_inf_mus', 'Haus_avg_sup_mus', 'Volume_all', 'Volume_lens', 'Volume_globe', 'Volume_nerve', 'Volume_int_fat', 'Volume_ext_fat', 'Volume_lat_mus', 'Volume_med_mus', 'Volume_inf_mus', 'Volume_sup_mus']]

Normality test

In [ ]:
# Shapiro-Wilk test for normality for each column of csv_file
from scipy.stats import shapiro

for i in range(1, len(csv_file.columns)):
    stat, p = shapiro(csv_file.iloc[:,i])
    print(f'{csv_file.columns[i]}: Statistics={stat}, p={p}')
    alpha = 0.05
    if p > alpha:
        print('Sample looks Gaussian (fail to reject H0)')
    else:
        print('Sample does not look Gaussian (reject H0)')
    # histogram plot
    plt.hist(csv_file.iloc[:,i])
    plt.show()

Outliers - IQR

In [ ]:
outliers_df1 = pd.DataFrame(columns=['Column', 'Outlier Subject'])
bounds_df = pd.DataFrame(columns=['Column', 'Lower Bound', 'Upper Bound']) # dataframe for bounds

# Calculate the IQR for each column
Q1 = csv_file.quantile(0.25)
Q3 = csv_file.quantile(0.75)
IQR = Q3 - Q1

# Find the outliers for each column
for column in csv_file.columns[1:]:
    lower_bound = Q1[column] - 1.5 * IQR[column]
    upper_bound = Q3[column] + 1.5 * IQR[column]

    # Append bounds to the bounds_df DataFrame
    new_row = pd.DataFrame({'Column': [column], 'Lower Bound': [lower_bound], 'Upper Bound': [upper_bound]})
    bounds_df = pd.concat([bounds_df, new_row], ignore_index=True)

    column_outliers = csv_file[(csv_file[column] < lower_bound) | (csv_file[column] > upper_bound)]
    new_row = pd.DataFrame({'Column': [column], 'Outlier Subject': [column_outliers['Subject'].values]})
    outliers_df1 = pd.concat([outliers_df1, new_row], ignore_index=True)

In [ ]:
# Initialize a new DataFrame with the same structure as csv_file but filled with False
outliers_df2 = pd.DataFrame(False, index=csv_file.index, columns=csv_file.columns)

# Copy the 'Subject' column from csv_file to outliers_df
outliers_df2['Subject'] = csv_file['Subject']

# Calculate the IQR for each column excluding 'Subject'
Q1 = csv_file.drop('Subject', axis=1).quantile(0.25)
Q3 = csv_file.drop('Subject', axis=1).quantile(0.75)
IQR = Q3 - Q1

# Find the outliers for each column excluding 'Subject'
for column in csv_file.columns:
    if column != 'Subject':
        lower_bound = Q1[column] - 1.5 * IQR[column]
        upper_bound = Q3[column] + 1.5 * IQR[column]
        # Update outliers_df with True for outliers
        outliers_df2[column] = (csv_file[column] < lower_bound) | (csv_file[column] > upper_bound)

# Replace all False values with ''
outliers_df2 = outliers_df2.replace(False, '')

In [ ]:
from collections import Counter

# make a list of 'Outlier Subject' without duplicates in row and column
outliers_list = []
for i in range(len(outliers_df1)):
    for j in range(len(outliers_df1.iloc[i,1])):
        outliers_list.append(outliers_df1.iloc[i,1][j])

# Count the occurrences of each outlier
outliers_counter = Counter(outliers_list)

# Keep only the outliers that appear more than once
outliers_list = [item for item, count in outliers_counter.items() if count > 1]

outliers_list = list(sorted(set(outliers_list)))

Reports to look at

In [ ]:
import pandas as pd

df_aux = pd.read_csv("/home/jaimebarranco/Desktop/MRI-QC/df_aux.csv")

# subdataframe of df_aux with the outliers_list
outliers_reports_df = df_aux[df_aux['subject'].isin(outliers_list)]
# add 'my_rate' column to outliers_reports_df
outliers_reports_df.loc[:, 'my_rate'] = 0.0
# order outliers_reports_df by 'report'
outliers_reports_df = outliers_reports_df.sort_values(by=['report'])
# save outliers_reports_df to excel file
# outliers_reports_df.to_excel("/home/jaimebarranco/Desktop/MRI-QC/qc2_atlas.xlsx", index=False)

#### Old and new subjects (N=69)

In [ ]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.pyplot import Line2D
import numpy as np

csv_file = pd.read_csv('/home/jaimebarranco/Desktop/new_manual_annotations/similarity/sim_manual_atlas_N43.csv')
csv_old = pd.read_csv('/mnt/sda1/Repos/a-eye/a-eye_preprocessing/ANTs/best_subjects_eye_cc/CustomTemplate_5_n1/sim_metrics_labels2subject3_N5.csv')

# group dataframes together
csv_file = pd.concat([csv_file, csv_old])
# remove four first rows (4 repeated subjects)
csv_file = csv_file.iloc[4:]

# Dataframes
data_dsc = [csv_file['DSC_all'], csv_file['DSC_lens'], csv_file['DSC_globe'], csv_file['DSC_nerve'], csv_file['DSC_int_fat'], csv_file['DSC_ext_fat'], csv_file['DSC_lat_mus'], csv_file['DSC_med_mus'], csv_file['DSC_inf_mus'], csv_file['DSC_sup_mus']]
data_ndsc = [csv_file['nDSC_all'],  csv_file['nDSC_lens'], csv_file['nDSC_globe'], csv_file['nDSC_nerve'], csv_file['nDSC_int_fat'], csv_file['nDSC_ext_fat'], csv_file['nDSC_lat_mus'], csv_file['nDSC_med_mus'], csv_file['nDSC_inf_mus'], csv_file['nDSC_sup_mus']]
data_vol = [csv_file['Volume_all'], csv_file['Volume_lens'], csv_file['Volume_globe'], csv_file['Volume_nerve'], csv_file['Volume_int_fat'], csv_file['Volume_ext_fat'], csv_file['Volume_lat_mus'], csv_file['Volume_med_mus'], csv_file['Volume_inf_mus'], csv_file['Volume_sup_mus']]
data_haus = [csv_file['Haus_avg_all'], csv_file['Haus_avg_lens'], csv_file['Haus_avg_globe'], csv_file['Haus_avg_nerve'], csv_file['Haus_avg_int_fat'], csv_file['Haus_avg_ext_fat'], csv_file['Haus_avg_lat_mus'], csv_file['Haus_avg_med_mus'], csv_file['Haus_avg_inf_mus'], csv_file['Haus_avg_sup_mus']]

labels = ['overall', 'lens', 'globe', 'nerve', 'intraconal fat', 'extraconal fat', 'lateral rectus muscle', 'medial rectus muscle', 'inferior rectus muscle', 'superior rectus muscle']
median = [np.around(np.median(x), 2) for x in data_dsc]
for i in range(len(labels)):
    print(labels[i], median[i])

# Figure
fig, axs = plt.subplots(3, figsize=(20,10), sharex=True)
fig.canvas.manager.set_window_title(f'Similarity metrics MANUAL vs ATLAS (N={len(csv_file)})')
fig.suptitle(f'Similarity metrics MANUAL vs ATLAS (N={len(csv_file)})')
fig.patch.set_facecolor('white')
# fig.tight_layout(pad=2)

# Set labels and titles
axs[0].set_title('DSC')
axs[0].set_ylim([0, 1])
axs[1].set_title('Hausdorff distance')
axs[1].set_ylim([0, 3])
axs[2].set_title('Volume difference')
axs[2].set_ylim([-2, 2])

# Outliers
flierprops = dict(markerfacecolor='0.9', markersize=2, linestyle='none')

# Boxplot & Swarmplot (points)
graph1 = sns.boxplot(data=data_dsc, ax=axs[0], flierprops=flierprops).set(ylabel="Value")
graph2 = sns.boxplot(data=data_haus, ax=axs[1], flierprops=flierprops).set(ylabel="Value")
graph3 = sns.boxplot(data=data_vol, ax=axs[2], flierprops=flierprops).set(ylabel="Value")
axs[2].set_xticklabels(['all','lens','globe','nerve','int_fat','ext_fat','lat_mus','med_mus','inf_mus','sup_mus'])

plt.show

# Save figure
# plt.savefig('/home/jaimebarranco/Desktop/new_manual_annotations/similarity/sim_manual_atlas_N69.png', bbox_inches='tight')

### nnUNet vs Manual

Number of files under 1kB in a specific folder

In [ ]:
# number of files under 1kB in a specific folder

import os
import glob

path = '/home/jaimebarranco/Desktop/new_manual_annotations/segmentations/nnunet'
files = glob.glob(path + '*.nii.gz')
bad_files = []

i=0
for file in sorted(files):
    if os.path.getsize(file) < 1000:
        subject = file.split('/')[-1].split('.')[0].split('_')[0]
        bad_files.append(subject)
        print(subject)
        i+=1

print(f'Number of images under 1kB: {i}')

Allignment

In [ ]:
import numpy as np
import nibabel as nb
import SimpleITK as sitk
import pandas as pd
import glob, os

reg_dir = '/home/jaimebarranco/Desktop/new_manual_annotations/segmentations/manual'
nnunet_dir = '/home/jaimebarranco/Desktop/new_manual_annotations/segmentations/nnunet'
output_dir = '/home/jaimebarranco/Desktop/new_manual_annotations/segmentations/nnunet_alligned'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

i=0
for file in sorted(glob.glob(nnunet_dir+'/*.nii.gz')):

    subject = os.path.basename(file).split('.')[0]
    # print(f'subject = {subject}')

    if os.path.exists(os.path.join(output_dir, f'{subject}.nii.gz')):
        continue
    else:
        # if subject == '2022160100236': # indent from here
        image_path_nnunet = f'{nnunet_dir}/{subject}.nii.gz'
        image_path_atlas = f'{reg_dir}/{subject}.nii.gz'

        # Load t1_aux (to allign the other)
        t1_aux = nb.load(image_path_atlas)
        # print(f't1_aux.affine: \n{t1_aux.affine}')
        # print(f't1_aux.header: \n{t1_aux.header}')

        # Load t1 image (not alligned)
        t1 = nb.load(image_path_nnunet)
        # print(f't1.affine: \n{t1.affine}')
        # print(f't1.header: \n{t1.header}')

        # Copy affine from t1_aux to t1
        nii = nb.Nifti1Image(t1.dataobj, t1_aux.affine, t1_aux.header)
        nii.to_filename(f'{output_dir}/{subject}.nii.gz')
        
        # Dealing with files in that folder
        # for f in glob.glob(base_dir+folder1+'/input/'+folder1+'_T1_oriented_hdr.nii.gz'):
        #     os.remove(f)

        i+=1
        # if (i==1):
        #     break

Compute sim metrics

In [ ]:
import SimpleITK as sitk
import numpy as np
import pandas as pd
import os
import nibabel as nb

# nDSC (normalized DSC)
def dice_norm_metric(ground_truth, predictions):
    '''
    For a single example returns DSC_norm, fpr, fnr
    '''

    # Reference for normalized DSC
    r = 0.001 # It should be 1/N*(np.sum(voxels_label[i])/np.sum(voxels_image[i])) i belonging to training set
    # Cast to float32 type
    gt = ground_truth.astype("float32")
    seg = predictions.astype("float32")
    im_sum = np.sum(seg) + np.sum(gt)
    if im_sum == 0:
        return 1.0, 1.0, 1.0
    else:
        if np.sum(gt) == 0:
            k = 1.0
        else:
            k = (1 - r) * np.sum(gt) / (r * (len(gt.flatten()) - np.sum(gt)))
        tp = np.sum(seg[gt == 1])
        fp = np.sum(seg[gt == 0])
        fn = np.sum(gt[seg == 0])
        fp_scaled = k * fp
        dsc_norm = 2 * tp / (fp_scaled + 2 * tp + fn)

        fpr = fp / (len(gt.flatten()) - np.sum(gt))
        if np.sum(gt) == 0:
            fnr = 1.0
        else:
            fnr = fn / np.sum(gt)
        return dsc_norm # fpr, fnr

# Paths - segmentation results
reg_dir = '/home/jaimebarranco/Desktop/new_manual_annotations/segmentations/manual'
nnunet_dir = '/home/jaimebarranco/Desktop/new_manual_annotations/segmentations/nnunet_alligned'

list_subjects = sorted(os.listdir(nnunet_dir))
list_subjects_clean = []

for file in list_subjects:
    # Name subject
    name_subject = file.split('.')[0]
    # only non-excluded subjects
    if name_subject not in list_qc1:
        print(f'subject: {name_subject}')
        list_subjects_clean.append(name_subject)
        continue

# List of volumes for each individual label and subject
len_path = len(list_subjects_clean) # elements in gt path = number of subjects
name_subject = [None]*len_path

# Save values in an array
# All labels
val_dsc             = np.zeros(len_path)
# val_hau             = np.zeros(len(rest_subjects))
val_hau_avg         = np.zeros(len_path)
val_vol             = np.zeros(len_path)
val_ndsc            = np.zeros(len_path)
# Lens
val_dsc_lens        = np.zeros(len_path)
val_hau_avg_lens    = np.zeros(len_path)
val_vol_lens        = np.zeros(len_path)
val_ndsc_lens       = np.zeros(len_path)
# Globe
val_dsc_globe       = np.zeros(len_path)
val_hau_avg_globe   = np.zeros(len_path)
val_vol_globe       = np.zeros(len_path)
val_ndsc_globe      = np.zeros(len_path)
# Optic nerve  
val_dsc_nerve       = np.zeros(len_path)
val_hau_avg_nerve   = np.zeros(len_path)
val_vol_nerve       = np.zeros(len_path)
val_ndsc_nerve      = np.zeros(len_path)
# Intraconal fat
val_dsc_int_fat     = np.zeros(len_path)
val_hau_avg_int_fat = np.zeros(len_path)
val_vol_int_fat     = np.zeros(len_path)
val_ndsc_int_fat    = np.zeros(len_path)
# Extraconal fat
val_dsc_ext_fat     = np.zeros(len_path)
val_hau_avg_ext_fat = np.zeros(len_path)
val_vol_ext_fat     = np.zeros(len_path)
val_ndsc_ext_fat    = np.zeros(len_path)
# Lateral rectus muscle
val_dsc_lat_mus     = np.zeros(len_path)
val_hau_avg_lat_mus = np.zeros(len_path)
val_vol_lat_mus     = np.zeros(len_path)
val_ndsc_lat_mus    = np.zeros(len_path)
# Medial rectus muscle
val_dsc_med_mus     = np.zeros(len_path)
val_hau_avg_med_mus = np.zeros(len_path)
val_vol_med_mus     = np.zeros(len_path)
val_ndsc_med_mus    = np.zeros(len_path)
# Inferior rectus muscle
val_dsc_inf_mus     = np.zeros(len_path)
val_hau_avg_inf_mus = np.zeros(len_path)
val_vol_inf_mus     = np.zeros(len_path)
val_ndsc_inf_mus    = np.zeros(len_path)
# Superior rectus muscle
val_dsc_sup_mus     = np.zeros(len_path)
val_hau_avg_sup_mus = np.zeros(len_path)
val_vol_sup_mus     = np.zeros(len_path)
val_ndsc_sup_mus    = np.zeros(len_path)

reader = sitk.ImageFileReader()

i=0
for file in list_subjects_clean:

    # Name subject
    name_subject[i] = file

    # bad files
    if name_subject[i] in bad_files:
        print('bad file')
        i+=1
        continue
    
    # MANUAL-based results
    reg_lab_path = f'{reg_dir}/{name_subject[i]}.nii.gz'
    reader.SetFileName(reg_lab_path)
    reg_lab_sitk = sitk.Cast(reader.Execute(), sitk.sitkUInt8)
    reg_lab_arr = sitk.GetArrayFromImage(reg_lab_sitk)

    # Deep Learning results
    dl_lab_path = f'{nnunet_dir}/{name_subject[i]}.nii.gz'
    # check if the file exists
    if not os.path.exists(dl_lab_path):
        print('file does not exist')
        i+=1
        continue
    reader.SetFileName(dl_lab_path)
    dl_lab_sitk = sitk.Cast(reader.Execute(), sitk.sitkUInt8)
    dl_lab_arr = sitk.GetArrayFromImage(dl_lab_sitk)
    
    # Image size
    im_lab_size = reg_lab_arr.shape[0]*reg_lab_arr.shape[1]*reg_lab_arr.shape[2]

    # LENS
    if np.count_nonzero(reg_lab_arr==1)==0 or np.count_nonzero(dl_lab_arr==1)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==1, dl_lab_sitk==1)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_lens[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_lens[i] = vol
        # Hausdorff distance
        hausdorf = sitk.HausdorffDistanceImageFilter()
        hausdorf.Execute(reg_lab_sitk==1, dl_lab_sitk==1)
        hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
        val_hau_avg_lens[i] = hausdorf_distance_avg
        # nDSC
        nDSC = dice_norm_metric(reg_lab_arr==1, dl_lab_arr==1)
        val_ndsc_lens[i] = nDSC
    
    # GLOBE EX LENS
    if np.count_nonzero(reg_lab_arr==2)==0 or np.count_nonzero(dl_lab_arr==2)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter 
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==2, dl_lab_sitk==2)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_globe[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_globe[i] = vol
        # Hausdorff distance
        hausdorf = sitk.HausdorffDistanceImageFilter()
        hausdorf.Execute(reg_lab_sitk==2, dl_lab_sitk==2)
        hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
        val_hau_avg_globe[i] = hausdorf_distance_avg
        # nDSC
        nDSC = dice_norm_metric(reg_lab_arr==2, dl_lab_arr==2)
        val_ndsc_globe[i] = nDSC

    # OPTIC NERVE
    if np.count_nonzero(reg_lab_arr==3)==0 or np.count_nonzero(dl_lab_arr==3)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter 
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==3, dl_lab_sitk==3)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_nerve[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_nerve[i] = vol
        # Hausdorff distance
        hausdorf = sitk.HausdorffDistanceImageFilter()
        hausdorf.Execute(reg_lab_sitk==3, dl_lab_sitk==3)
        hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
        val_hau_avg_nerve[i] = hausdorf_distance_avg
        # nDSC
        nDSC = dice_norm_metric(reg_lab_arr==3, dl_lab_arr==3)
        val_ndsc_nerve[i] = nDSC

    # INTRACONAL FAT
    if np.count_nonzero(reg_lab_arr==4)==0 or np.count_nonzero(dl_lab_arr==4)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter 
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==4, dl_lab_sitk==4)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_int_fat[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_int_fat[i] = vol
        # Hausdorff distance
        hausdorf = sitk.HausdorffDistanceImageFilter()
        hausdorf.Execute(reg_lab_sitk==4, dl_lab_sitk==4)
        hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
        val_hau_avg_int_fat[i] = hausdorf_distance_avg
        # nDSC
        nDSC = dice_norm_metric(reg_lab_arr==4, dl_lab_arr==4)
        val_ndsc_int_fat[i] = nDSC

    # EXTRACONAL FAT
    if np.count_nonzero(reg_lab_arr==5)==0 or np.count_nonzero(dl_lab_arr==5)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter 
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==5, dl_lab_sitk==5)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_ext_fat[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_ext_fat[i] = vol
        # Hausdorff distance
        hausdorf = sitk.HausdorffDistanceImageFilter()
        hausdorf.Execute(reg_lab_sitk==5, dl_lab_sitk==5)
        hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
        val_hau_avg_ext_fat[i] = hausdorf_distance_avg
        # nDSC
        nDSC = dice_norm_metric(reg_lab_arr==5, dl_lab_arr==5)
        val_ndsc_ext_fat[i] = nDSC

    # LATERAL RECTUS MUSCLE
    if np.count_nonzero(reg_lab_arr==6)==0 or np.count_nonzero(dl_lab_arr==6)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==6, dl_lab_sitk==6)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_lat_mus[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_lat_mus[i] = vol
        # Hausdorff distance
        hausdorf = sitk.HausdorffDistanceImageFilter()
        hausdorf.Execute(reg_lab_sitk==6, dl_lab_sitk==6)
        hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
        val_hau_avg_lat_mus[i] = hausdorf_distance_avg
        # nDSC
        nDSC = dice_norm_metric(reg_lab_arr==6, dl_lab_arr==6)
        val_ndsc_lat_mus[i] = nDSC

    # MEDIAL RECTUS MUSCLE
    if np.count_nonzero(reg_lab_arr==7)==0 or np.count_nonzero(dl_lab_arr==7)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter 
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==7, dl_lab_sitk==7)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_med_mus[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_med_mus[i] = vol
        # Hausdorff distance
        if np.count_nonzero(reg_lab_arr==7)==0 or np.count_nonzero(dl_lab_arr==7)==0: val_hau_avg_lens[i] = 100
        else :
            hausdorf = sitk.HausdorffDistanceImageFilter()
            hausdorf.Execute(reg_lab_sitk==7, dl_lab_sitk==7)
            hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
            val_hau_avg_med_mus[i] = hausdorf_distance_avg
        # nDSC
        nDSC = dice_norm_metric(reg_lab_arr==7, dl_lab_arr==7)
        val_ndsc_med_mus[i] = nDSC

    # INFERIOR RECTUS MUSCLE
    if np.count_nonzero(reg_lab_arr==8)==0 or np.count_nonzero(dl_lab_arr==8)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==8, dl_lab_sitk==8)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_inf_mus[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_inf_mus[i] = vol
        # Hausdorff distance
        if np.count_nonzero(reg_lab_arr==8)==0 or np.count_nonzero(dl_lab_arr==8)==0: val_hau_avg_lens[i] = 100
        else :
            hausdorf = sitk.HausdorffDistanceImageFilter()
            hausdorf.Execute(reg_lab_sitk==8, dl_lab_sitk==8)
            hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
            val_hau_avg_inf_mus[i] = hausdorf_distance_avg
        # nDSC
        nDSC = dice_norm_metric(reg_lab_arr==8, dl_lab_arr==8)
        val_ndsc_inf_mus[i] = nDSC

    # SUPERIOR RECTUS MUSCLE
    if np.count_nonzero(reg_lab_arr==9)==0 or np.count_nonzero(dl_lab_arr==9)==0: # if there is no label in the image
        val_hau_avg_lens[i] = 100 # set hausdorff distance to 100 [-inf, inf]
        val_dsc_lens[i] = 0 # set dice coefficient to 0 [0, 1]
        val_vol_lens[i] = 2 # set volume similarity to 2 [-2, 2]
        val_ndsc_lens[i] = 0 # set normalized dice coefficient to 0 [0, 1]
    else:
        # Measures Image Filter
        overlap_measures_filter = sitk.LabelOverlapMeasuresImageFilter()
        overlap_measures_filter.Execute(reg_lab_sitk==9, dl_lab_sitk==9)
        # DSC
        dsc = overlap_measures_filter.GetDiceCoefficient() # Get the mean overlap (Dice coefficient) over all labels
        val_dsc_sup_mus[i] = dsc
        # Volume
        vol = overlap_measures_filter.GetVolumeSimilarity() # Get the volume similarity over all labels
        val_vol_sup_mus[i] = vol
        # Hausdorff distance
        hausdorf = sitk.HausdorffDistanceImageFilter()
        hausdorf.Execute(reg_lab_sitk==9, dl_lab_sitk==9)
        hausdorf_distance_avg = hausdorf.GetAverageHausdorffDistance() # Return the computed Hausdorff distance
        val_hau_avg_sup_mus[i] = hausdorf_distance_avg
        # nDSCvol
        nDSC = dice_norm_metric(reg_lab_arr==9, dl_lab_arr==9)
        val_ndsc_sup_mus[i] = nDSC

    # ALL LABELS
    # DSC
    dsc = (val_dsc_lens[i]+val_dsc_globe[i]+val_dsc_nerve[i]+val_dsc_int_fat[i]+val_dsc_ext_fat[i]+val_dsc_lat_mus[i]+val_dsc_med_mus[i]+val_dsc_inf_mus[i]+val_dsc_sup_mus[i])/9
    val_dsc[i] = dsc
    # Volume
    vol = (val_vol_lens[i]+val_vol_globe[i]+val_vol_nerve[i]+val_vol_int_fat[i]+val_vol_ext_fat[i]+val_vol_lat_mus[i]+val_vol_med_mus[i]+val_vol_inf_mus[i]+val_vol_sup_mus[i])/9
    val_vol[i] = vol
    # Hausdorff distance
    hau_avg = (val_hau_avg_lens[i]+val_hau_avg_globe[i]+val_hau_avg_nerve[i]+val_hau_avg_int_fat[i]+val_hau_avg_ext_fat[i]+val_hau_avg_lat_mus[i]+val_hau_avg_med_mus[i]+val_hau_avg_inf_mus[i]+val_hau_avg_sup_mus[i])/9
    val_hau_avg[i] = hau_avg
    # nDSC
    nDSC = (val_ndsc_lens[i]+val_ndsc_globe[i]+val_ndsc_nerve[i]+val_ndsc_int_fat[i]+val_ndsc_ext_fat[i]+val_ndsc_lat_mus[i]+val_ndsc_med_mus[i]+val_ndsc_inf_mus[i]+val_ndsc_sup_mus[i])/9
    val_ndsc[i] = nDSC

    i+=1
    # if i==1:
    #     break

Write values to csv

In [ ]:
import csv

metrics = [
    'Subject',  'DSC_all',     'Haus_avg_all',     'Volume_all',     'nDSC_all', 
                'DSC_lens',    'Haus_avg_lens',    'Volume_lens',    'nDSC_lens',  
                'DSC_globe',   'Haus_avg_globe',   'Volume_globe',   'nDSC_globe', 
                'DSC_nerve',   'Haus_avg_nerve',   'Volume_nerve',   'nDSC_nerve', 
                'DSC_int_fat', 'Haus_avg_int_fat', 'Volume_int_fat', 'nDSC_int_fat',
                'DSC_ext_fat', 'Haus_avg_ext_fat', 'Volume_ext_fat', 'nDSC_ext_fat',
                'DSC_lat_mus', 'Haus_avg_lat_mus', 'Volume_lat_mus', 'nDSC_lat_mus',
                'DSC_med_mus', 'Haus_avg_med_mus', 'Volume_med_mus', 'nDSC_med_mus',
                'DSC_inf_mus', 'Haus_avg_inf_mus', 'Volume_inf_mus', 'nDSC_inf_mus',
                'DSC_sup_mus', 'Haus_avg_sup_mus', 'Volume_sup_mus', 'nDSC_sup_mus'
]

vals = np.array([
    name_subject,   val_dsc,         val_hau_avg,         val_vol,         val_ndsc,        
                    val_dsc_lens,    val_hau_avg_lens,    val_vol_lens,    val_ndsc_lens,   
                    val_dsc_globe,   val_hau_avg_globe,   val_vol_globe,   val_ndsc_globe,  
                    val_dsc_nerve,   val_hau_avg_nerve,   val_vol_nerve,   val_ndsc_nerve,  
                    val_dsc_int_fat, val_hau_avg_int_fat, val_vol_int_fat, val_ndsc_int_fat,
                    val_dsc_ext_fat, val_hau_avg_ext_fat, val_vol_ext_fat, val_ndsc_ext_fat,
                    val_dsc_lat_mus, val_hau_avg_lat_mus, val_vol_lat_mus, val_ndsc_lat_mus,
                    val_dsc_med_mus, val_hau_avg_med_mus, val_vol_med_mus, val_ndsc_med_mus,
                    val_dsc_inf_mus, val_hau_avg_inf_mus, val_vol_inf_mus, val_ndsc_inf_mus,
                    val_dsc_sup_mus, val_hau_avg_sup_mus, val_vol_sup_mus, val_ndsc_sup_mus,
])

vals = vals.T

with open('/home/jaimebarranco/Desktop/new_manual_annotations/similarity/sim_manual_nnunet_N43.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow(metrics)
    writer.writerows(vals)

Read csv

In [ ]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.pyplot import Line2D
import numpy as np

csv_file = pd.read_csv('/home/jaimebarranco/Desktop/new_manual_annotations/similarity/sim_manual_nnunet_N43.csv')
num_subjects = len(csv_file)
csv_file = csv_file[~csv_file['Subject'].isin([int(x) for x in bad_files])].reset_index(drop=True)

# Dataframes {DSC, nDSC, Volume (voxels)} separate labels for N=5 only
data_dsc = [csv_file['DSC_all'], csv_file['DSC_lens'], csv_file['DSC_globe'], csv_file['DSC_nerve'], csv_file['DSC_int_fat'], csv_file['DSC_ext_fat'], csv_file['DSC_lat_mus'], csv_file['DSC_med_mus'], csv_file['DSC_inf_mus'], csv_file['DSC_sup_mus']]
data_ndsc = [csv_file['nDSC_all'],  csv_file['nDSC_lens'], csv_file['nDSC_globe'], csv_file['nDSC_nerve'], csv_file['nDSC_int_fat'], csv_file['nDSC_ext_fat'], csv_file['nDSC_lat_mus'], csv_file['nDSC_med_mus'], csv_file['nDSC_inf_mus'], csv_file['nDSC_sup_mus']]
data_vol = [csv_file['Volume_all'], csv_file['Volume_lens'], csv_file['Volume_globe'], csv_file['Volume_nerve'], csv_file['Volume_int_fat'], csv_file['Volume_ext_fat'], csv_file['Volume_lat_mus'], csv_file['Volume_med_mus'], csv_file['Volume_inf_mus'], csv_file['Volume_sup_mus']]
data_haus = [csv_file['Haus_avg_all'], csv_file['Haus_avg_lens'], csv_file['Haus_avg_globe'], csv_file['Haus_avg_nerve'], csv_file['Haus_avg_int_fat'], csv_file['Haus_avg_ext_fat'], csv_file['Haus_avg_lat_mus'], csv_file['Haus_avg_med_mus'], csv_file['Haus_avg_inf_mus'], csv_file['Haus_avg_sup_mus']]

labels = ['overall', 'lens', 'globe', 'nerve', 'intraconal fat', 'extraconal fat', 'lateral rectus muscle', 'medial rectus muscle', 'inferior rectus muscle', 'superior rectus muscle']
median = [np.around(np.median(x), 2) for x in data_dsc]
print('median DSC:')
for i in range(len(labels)):
    print(labels[i], median[i])

In [ ]:
# Figure
fig, axs = plt.subplots(3, figsize=(20,10), sharex=True)
fig.canvas.manager.set_window_title(f'Similarity metrics MANUAL vs nnUNet (N={num_subjects})')
fig.suptitle(f'Similarity metrics MANUAL vs nnUNet (N={num_subjects})')
fig.patch.set_facecolor('white')
# fig.tight_layout(pad=2)

# Set labels and titles
axs[0].set_title('DSC')
axs[0].set_ylim([0, 1])
axs[1].set_title('Hausdorff distance')
axs[1].set_ylim([0, 3])
axs[2].set_title('Volume difference')
axs[2].set_ylim([-2, 2])

# Outliers
flierprops = dict(markerfacecolor='0.9', markersize=2, linestyle='none')

# Boxplot & Swarmplot (points)
graph1 = sns.boxplot(data=data_dsc, ax=axs[0], flierprops=flierprops).set(ylabel="Value")
graph2 = sns.boxplot(data=data_haus, ax=axs[1], flierprops=flierprops).set(ylabel="Value")
graph3 = sns.boxplot(data=data_vol, ax=axs[2], flierprops=flierprops).set(ylabel="Value")
axs[2].set_xticklabels(['all','lens','globe','nerve','int_fat','ext_fat','lat_mus','med_mus','inf_mus','sup_mus'])

# Save figure
# plt.savefig('/home/jaimebarranco/Desktop/new_manual_annotations/similarity/sim_manual_nnunet_N43.png', bbox_inches='tight')

plt.show

#### Outliers

In [ ]:
# remove nDSC columns from csv_file
csv_file = csv_file.drop(columns=['nDSC_all', 'nDSC_lens', 'nDSC_globe', 'nDSC_nerve', 'nDSC_int_fat', 'nDSC_ext_fat', 'nDSC_lat_mus', 'nDSC_med_mus', 'nDSC_inf_mus', 'nDSC_sup_mus'])
# reorder columns, first the DSCs, then the Hausdorff distances, then the volumes
csv_file = csv_file[['Subject', 'DSC_all', 'DSC_lens', 'DSC_globe', 'DSC_nerve', 'DSC_int_fat', 'DSC_ext_fat', 'DSC_lat_mus', 'DSC_med_mus', 'DSC_inf_mus', 'DSC_sup_mus', 'Haus_avg_all', 'Haus_avg_lens', 'Haus_avg_globe', 'Haus_avg_nerve', 'Haus_avg_int_fat', 'Haus_avg_ext_fat', 'Haus_avg_lat_mus', 'Haus_avg_med_mus', 'Haus_avg_inf_mus', 'Haus_avg_sup_mus', 'Volume_all', 'Volume_lens', 'Volume_globe', 'Volume_nerve', 'Volume_int_fat', 'Volume_ext_fat', 'Volume_lat_mus', 'Volume_med_mus', 'Volume_inf_mus', 'Volume_sup_mus']]

Normality test

In [ ]:
# Shapiro-Wilk test for normality for each column of csv_file
from scipy.stats import shapiro

for i in range(1, len(csv_file.columns)):
    stat, p = shapiro(csv_file.iloc[:,i])
    print(f'{csv_file.columns[i]}: Statistics={stat}, p={p}')
    alpha = 0.05
    if p > alpha:
        print('Sample looks Gaussian (fail to reject H0)')
    else:
        print('Sample does not look Gaussian (reject H0)')
    # histogram plot
    plt.hist(csv_file.iloc[:,i])
    plt.show()

Outliers - IQR

In [ ]:
outliers_df1 = pd.DataFrame(columns=['Column', 'Outlier Subject'])
bounds_df = pd.DataFrame(columns=['Column', 'Lower Bound', 'Upper Bound']) # dataFrame for bounds

# Calculate the IQR for each column
Q1 = csv_file.quantile(0.25)
Q3 = csv_file.quantile(0.75)
IQR = Q3 - Q1

# Find the outliers for each column
for column in csv_file.columns[1:]:
    lower_bound = Q1[column] - 1.5 * IQR[column]
    upper_bound = Q3[column] + 1.5 * IQR[column]

    # Append bounds to the bounds_df DataFrame
    new_row = pd.DataFrame({'Column': [column], 'Lower Bound': [lower_bound], 'Upper Bound': [upper_bound]})
    bounds_df = pd.concat([bounds_df, new_row], ignore_index=True)

    column_outliers = csv_file[(csv_file[column] < lower_bound) | (csv_file[column] > upper_bound)]
    new_row = pd.DataFrame({'Column': [column], 'Outlier Subject': [column_outliers['Subject'].values]})
    outliers_df1 = pd.concat([outliers_df1, new_row], ignore_index=True)

In [ ]:
# Initialize a new DataFrame with the same structure as csv_file but filled with False
outliers_df2 = pd.DataFrame(False, index=csv_file.index, columns=csv_file.columns)

# Copy the 'Subject' column from csv_file to outliers_df
outliers_df2['Subject'] = csv_file['Subject']

# Calculate the IQR for each column excluding 'Subject'
Q1 = csv_file.drop('Subject', axis=1).quantile(0.25)
Q3 = csv_file.drop('Subject', axis=1).quantile(0.75)
IQR = Q3 - Q1

# Find the outliers for each column excluding 'Subject'
for column in csv_file.columns:
    if column != 'Subject':
        lower_bound = Q1[column] - 1.5 * IQR[column]
        upper_bound = Q3[column] + 1.5 * IQR[column]
        # Update outliers_df with True for outliers
        outliers_df2[column] = (csv_file[column] < lower_bound) | (csv_file[column] > upper_bound)

# Replace all False values with ''
outliers_df2 = outliers_df2.replace(False, '')

In [ ]:
from collections import Counter

# make a list of 'Outlier Subject' without duplicates in row and column
outliers_list = []
for i in range(len(outliers_df1)):
    for j in range(len(outliers_df1.iloc[i,1])):
        outliers_list.append(outliers_df1.iloc[i,1][j])

# Count the occurrences of each outlier
outliers_counter = Counter(outliers_list)

# Keep only the outliers that appear more than once
outliers_list = [item for item, count in outliers_counter.items() if count > 1]

outliers_list = list(sorted(set(outliers_list)))

Reports to look at - removing already analysed subjects from atlas qc2

In [ ]:
import pandas as pd

qc2_atlas = pd.read_excel("/home/jaimebarranco/Desktop/MRI-QC/qc2_atlas.xlsx")

In [ ]:
import pandas as pd

df_aux = pd.read_csv("/home/jaimebarranco/Desktop/MRI-QC/df_aux.csv")

# subdataframe of df_aux with the outliers_list
outliers_reports_df = df_aux[df_aux['subject'].isin(outliers_list)]
# add 'my_rate' column to outliers_reports_df
outliers_reports_df.loc[:, 'my_rate'] = 0.0
# order outliers_reports_df by 'report'
outliers_reports_df = outliers_reports_df.sort_values(by=['report'])
# remove subjects from outliers_reports_df that are in qc2_atlas
# outliers_reports_df = outliers_reports_df[~outliers_reports_df['subject'].isin(qc2_atlas['subject'])]
# save outliers_reports_df to excel file
# outliers_reports_df.to_excel("/home/jaimebarranco/Desktop/MRI-QC/qc2_nnunet.xlsx", index=False)

Joint outliers

In [ ]:
import pandas as pd

qc2_atlas = pd.read_excel("/home/jaimebarranco/Desktop/MRI-QC/qc2_atlas.xlsx")
qc2_nnunet = pd.read_excel("/home/jaimebarranco/Desktop/MRI-QC/qc2_nnunet.xlsx")

# joint dataframes
qc2 = pd.concat([qc2_atlas, qc2_nnunet], ignore_index=True)
# keep only the rows with '0' in my_rate column
qc2 = qc2[qc2['my_rate'] == 0]
# save it to .csv
qc2.to_csv("/home/jaimebarranco/Desktop/MRI-QC/qc2.csv", index=False)

### ATLAS vs nnUNet

Read csv

In [ ]:
import pandas as pd
import numpy as np

# CSVs
csv_sm_reg = '/home/jaimebarranco/Desktop/new_manual_annotations/similarity/sim_manual_atlas_N43.csv'
csv_sm_nn = '/home/jaimebarranco/Desktop/new_manual_annotations/similarity/sim_manual_nnunet_N43.csv'

# Pandas read CSV
df_reg = pd.read_csv(csv_sm_reg)
df_nn = pd.read_csv(csv_sm_nn)

# Gather data from generated dataframes
df_dsc = np.array([
    df_reg['DSC_all'], df_nn['DSC_all'],
    df_reg['DSC_lens'], df_nn['DSC_lens'], 
    df_reg['DSC_globe'], df_nn['DSC_globe'], 
    df_reg['DSC_nerve'], df_nn['DSC_nerve'], 
    df_reg['DSC_int_fat'], df_nn['DSC_int_fat'], 
    df_reg['DSC_ext_fat'], df_nn['DSC_ext_fat'], 
    df_reg['DSC_lat_mus'], df_nn['DSC_lat_mus'], 
    df_reg['DSC_med_mus'], df_nn['DSC_med_mus'], 
    df_reg['DSC_inf_mus'], df_nn['DSC_inf_mus'], 
    df_reg['DSC_sup_mus'], df_nn['DSC_sup_mus']])
df_hd = np.array([
    df_reg['Haus_avg_all'], df_nn['Haus_avg_all'], 
    df_reg['Haus_avg_lens'], df_nn['Haus_avg_lens'], 
    df_reg['Haus_avg_globe'], df_nn['Haus_avg_globe'], 
    df_reg['Haus_avg_nerve'], df_nn['Haus_avg_nerve'], 
    df_reg['Haus_avg_int_fat'], df_nn['Haus_avg_int_fat'], 
    df_reg['Haus_avg_ext_fat'], df_nn['Haus_avg_ext_fat'], 
    df_reg['Haus_avg_lat_mus'], df_nn['Haus_avg_lat_mus'], 
    df_reg['Haus_avg_med_mus'], df_nn['Haus_avg_med_mus'], 
    df_reg['Haus_avg_inf_mus'], df_nn['Haus_avg_inf_mus'], 
    df_reg['Haus_avg_sup_mus'], df_nn['Haus_avg_sup_mus']])
df_vol = np.array([
    df_reg['Volume_all'], df_nn['Volume_all'], 
    df_reg['Volume_lens'], df_nn['Volume_lens'], 
    df_reg['Volume_globe'], df_nn['Volume_globe'], 
    df_reg['Volume_nerve'], df_nn['Volume_nerve'], 
    df_reg['Volume_int_fat'], df_nn['Volume_int_fat'], 
    df_reg['Volume_ext_fat'], df_nn['Volume_ext_fat'], 
    df_reg['Volume_lat_mus'], df_nn['Volume_lat_mus'], 
    df_reg['Volume_med_mus'], df_nn['Volume_med_mus'], 
    df_reg['Volume_inf_mus'], df_nn['Volume_inf_mus'], 
    df_reg['Volume_sup_mus'], df_nn['Volume_sup_mus']])

Plot

In [ ]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.pyplot import Line2D
import numpy as np

# Figure
fig, axs = plt.subplots(3, sharex=True, figsize=(28,20))
fig.canvas.manager.set_window_title(f'Similarity metrics ATLAS vs nnUNet (N={len(df_reg)})')
fig.suptitle(f'Similarity metrics ATLAS vs nnUNet (N={len(df_reg)})')
fig.patch.set_facecolor('white')
fig.tight_layout(pad=3)

# Legend
colours_palette = sns.color_palette("Blues", 2)
legend_elements = [Line2D([0], [0], color=colours_palette[0], lw=4, label='Atlas-based'),
    Line2D([0], [0], color=colours_palette[1], lw=4, label='nnUNet'),
    Line2D([], [], color='black', label='mean', marker='+', markersize=5, linestyle='None')]
axs[0].legend(handles=legend_elements)

# Axis
axs[0].set_title('DSC')
axs[0].set_ylim([0, 1])
axs[1].set_title('Hausdorff distance')
axs[1].set_ylim([0, 3])
axs[2].set_title('Volume difference')
axs[2].set_ylim([-2, 2])

# Boxplots
# colours_palette = ['green','purple','green','purple','green','purple','green','purple','green','purple','green','purple','green','purple','green','purple','green','purple','green','purple']
meanpointprops = {"marker":"+", "markerfacecolor":"black", "markeredgecolor":"black", "markersize":"10"}
graph1 = sns.boxplot(data=df_dsc.T, ax=axs[0], palette=colours_palette, showmeans=True, meanprops=meanpointprops).set(ylabel="Value")
graph2 = sns.boxplot(data=df_hd.T, ax=axs[1], palette=colours_palette, showmeans=True, meanprops=meanpointprops).set(ylabel="Value")
graph3 = sns.boxplot(data=df_vol.T, ax=axs[2], palette=colours_palette, showmeans=True, meanprops=meanpointprops).set(ylabel="Value")
# ax = sns.swarmplot(data=df_dsc)

# axs[0].set_xticklabels([
#     f'all \n{np.around(np.median(df_dsc[0]), 3)}', f'all \n{np.around(np.median(df_dsc[1]), 3)}',
#     f'lens \n{np.around(np.median(df_dsc[2]), 3)}', f'lens \n{np.around(np.median(df_dsc[3]), 3)}',
#     f'globe \n{np.around(np.median(df_dsc[4]), 3)}', f'globe \n{np.around(np.median(df_dsc[5]), 3)}',
#     f'nerve \n{np.around(np.median(df_dsc[6]), 3)}', f'nerve \n{np.around(np.median(df_dsc[7]), 3)}',
#     f'int_fat \n{np.around(np.median(df_dsc[8]), 3)}', f'int_fat \n{np.around(np.median(df_dsc[9]), 3)}',
#     f'ext_fat \n{np.around(np.median(df_dsc[10]), 3)}', f'ext_fat \n{np.around(np.median(df_dsc[11]), 3)}',
#     f'lat_mus \n{np.around(np.median(df_dsc[12]), 3)}', f'lat_mus \n{np.around(np.median(df_dsc[13]), 3)}',
#     f'med_mus \n{np.around(np.median(df_dsc[14]), 3)}', f'med_mus \n{np.around(np.median(df_dsc[15]), 3)}',
#     f'inf_mus \n{np.around(np.median(df_dsc[16]), 3)}', f'inf_mus \n{np.around(np.median(df_dsc[17]), 3)}',
#     f'sup_mus \n{np.around(np.median(df_dsc[18]), 3)}', f'sup_mus \n{np.around(np.median(df_dsc[19]), 3)}'])

axs[0].set_xticklabels([
    f'all', f'all',
    f'lens', f'lens',
    f'globe', f'globe',
    f'nerve', f'nerve',
    f'int_fat', f'int_fat',
    f'ext_fat', f'ext_fat',
    f'lat_mus', f'lat_mus',
    f'med_mus', f'med_mus',
    f'inf_mus', f'inf_mus',
    f'sup_mus', f'sup_mus'])

# plt.savefig(f'/home/jaimebarranco/Desktop/new_manual_annotations/similarity/sim_atlas_vs_nn_N43.png', bbox_inches='tight', dpi=300)

plt.show()

#### Pair-wise comparisons

Shapiro-Wilk test for Normality

In [ ]:
from scipy.stats import shapiro

# List to hold p-values for each metric
p_values_shapiro = []

# Shapiro-Wilk test for normality for each metric
for metric_data in [df_dsc, df_hd, df_vol]:
    for data in metric_data:
        # Shapiro-Wilk test
        stat, p_val = shapiro(data)
        p_values_shapiro.append(p_val)

# Check normality based on p-values (considering 0.05 significance level)
is_normal = [p > 0.05 for p in p_values_shapiro]

In [ ]:
# list labels DSC
labels_DSC = ['DSC_all_reg', 'DSC_all_nn', 'DSC_lens_reg', 'DSC_lens_nn', 'DSC_globe_reg', 'DSC_globe_nn', 'DSC_nerve_reg', 'DSC_nerve_nn', 'DSC_int_fat_reg', 'DSC_int_fat_nn', 'DSC_ext_fat_reg', 'DSC_ext_fat_nn', 'DSC_lat_mus_reg', 'DSC_lat_mus_nn', 'DSC_med_mus_reg', 'DSC_med_mus_nn', 'DSC_inf_mus_reg', 'DSC_inf_mus_nn', 'DSC_sup_mus_reg', 'DSC_sup_mus_nn']
# list labels Hausdorff distance
labels_hd = ['HD_all_reg', 'HD_all_nn', 'HD_lens_reg', 'HD_lens_nn', 'HD_globe_reg', 'HD_globe_nn', 'HD_nerve_reg', 'HD_nerve_nn', 'HD_int_fat_reg', 'HD_int_fat_nn', 'HD_ext_fat_reg', 'HD_ext_fat_nn', 'HD_lat_mus_reg', 'HD_lat_mus_nn', 'HD_med_mus_reg', 'HD_med_mus_nn', 'HD_inf_mus_reg', 'HD_inf_mus_nn', 'HD_sup_mus_reg', 'HD_sup_mus_nn']
# list labels Volume difference
labels_vol = ['VD_all_reg', 'VD_all_nn', 'VD_lens_reg', 'VD_lens_nn', 'VD_globe_reg', 'VD_globe_nn', 'VD_nerve_reg', 'VD_nerve_nn', 'VD_int_fat_reg', 'VD_int_fat_nn', 'VD_ext_fat_reg', 'VD_ext_fat_nn', 'VD_lat_mus_reg', 'VD_lat_mus_nn', 'VD_med_mus_reg', 'VD_med_mus_nn', 'VD_inf_mus_reg', 'VD_inf_mus_nn', 'VD_sup_mus_reg', 'VD_sup_mus_nn']

# concatenate labels
labels = labels_DSC + labels_hd + labels_vol

# dataframe with labels and is_normal
df_normal = pd.DataFrame({'labels': labels, 'is_normal': is_normal})

Comparison tests: Wilcoxon or Student's t-test

In [ ]:
from scipy.stats import mannwhitneyu, ttest_ind, ttest_rel, wilcoxon

# For independent subjects: mannwhitneyu if not normal and ttest_ind if normal
# For metrics on the same subject (our case): ttes_rel if normal and wilcoxon if not normal

# List to hold u-values, p-values
t_values_test = []
u_values_test = []
p_values_test = []

# Perform comparison tests for each metric
for metric_data in [df_dsc, df_hd, df_vol]:
    for i in range(0, len(metric_data), 2):
        method_1 = metric_data[i]
        method_2 = metric_data[i + 1]
        
        if is_normal[i] and is_normal[i + 1]:
            # If both are normal, use Student's t-test
            t_val, p_val = ttest_rel(method_1, method_2) # ttest_ind, ttest_rel
            t_values_test.append(t_val)
        else:
            # If not normal, use Mann-Whitney U test
            u_val, p_val = wilcoxon(method_1, method_2) # mannwhitneyu, wilcoxon
            u_values_test.append(u_val)
        
        p_values_test.append(p_val)

# significant p-values
is_significant = [p < 0.05 for p in p_values_test]

In [ ]:
# list pair-labels DSC
pair_labels_DSC = ['DSC_all', 'DSC_lens', 'DSC_globe', 'DSC_nerve', 'DSC_int_fat', 'DSC_ext_fat', 'DSC_lat_mus', 'DSC_med_mus', 'DSC_inf_mus', 'DSC_sup_mus']
# list labels Hausdorff distance
pair_labels_hd = ['HD_all', 'HD_lens', 'HD_globe', 'HD_nerve', 'HD_int_fat', 'HD_ext_fat', 'HD_lat_mus', 'HD_med_mus', 'HD_inf_mus', 'HD_sup_mus']
# list labels Volume difference
pair_labels_vol = ['VD_all', 'VD_lens', 'VD_globe', 'VD_nerve', 'VD_int_fat', 'VD_ext_fat', 'VD_lat_mus', 'VD_med_mus', 'VD_inf_mus', 'VD_sup_mus']

# concatenate labels
pair_labels = pair_labels_DSC + pair_labels_hd + pair_labels_vol

# dataframe with labels and is_significant
df_significant = pd.DataFrame({'labels': pair_labels, 'is_significant': is_significant})

FDR correction (False Discovery Rate) - Benjamini-Hochberg

In [ ]:
from statsmodels.stats.multitest import multipletests

# Apply Benjamini-Hochberg procedure
rejected, adjusted_p_values, _, _ = multipletests(p_values_test, method='fdr_bh')

is_significant_fdr = [p < 0.05 for p in adjusted_p_values]

# Find significant p-values after FDR correction
significant_indices = [i for i, rej in enumerate(rejected) if rej]

# Retrieve the significant p-values after FDR correction
significant_p_values_fdr = [adjusted_p_values[i] for i in significant_indices]

In [ ]:
# dataframe with labels and is_significant
df_significant_fdr = pd.DataFrame({'labels': pair_labels, 'is_significant_fdr': is_significant_fdr})

Cohen's d values

In [ ]:
from scipy.stats import t

# Function to calculate confidence interval for Cohen's d
def cohen_d_confidence_interval(x, y, alpha=0.05):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2  # Degrees of freedom

    # Calculate Cohen's d
    d = (np.mean(x) - np.mean(y)) / np.sqrt(((nx - 1) * np.var(x, ddof=1) + (ny - 1) * np.var(y, ddof=1)) / dof)
    
    # Calculate pooled standard deviation
    pooled_var = ((nx - 1) * np.var(x, ddof=1) + (ny - 1) * np.var(y, ddof=1)) / dof
    
    # Calculate standard error
    se = np.sqrt(pooled_var * (1 / nx + 1 / ny))
    
    # Calculate t-value for the confidence interval
    t_val = t.ppf(1 - alpha / 2, dof)
    
    # Calculate margin of error
    margin_error = t_val * se
    
    # Calculate lower and upper bounds of the confidence interval
    lower_bound = d - margin_error
    upper_bound = d + margin_error
    
    return d, lower_bound, upper_bound

# List to hold the d values and the confidence intervals
d_values = []
confidence_intervals_low = []
confidence_intervals_up = []
effect = []

# Perform comparison for each metric
for metric_data in [df_dsc, df_hd, df_vol]:
    for i in range(0, len(metric_data), 2):
        method_1 = metric_data[i]
        method_2 = metric_data[i + 1]

        d, ci_low, ci_up = cohen_d_confidence_interval(method_1, method_2)

        d_values.append(d)
        confidence_intervals_low.append(ci_low)
        confidence_intervals_up.append(ci_up)

        if np.abs(d) < 0.5:
            effect.append('low')
        elif np.abs(d) > 0.5 and np.abs(d) < 0.8:
            effect.append('medium')
        else:
            effect.append('strong')

In [ ]:
# dataframe with labels, d_values and confidence intervals
df_cohen_d = pd.DataFrame({'labels': pair_labels, 'd_values': d_values, 'confidence_intervals_low': confidence_intervals_low, 'confidence_intervals_up': confidence_intervals_up, 'effect': effect})

Cohen's d values bootstrap

In [ ]:
"""
Bootstrap sampling N=5000
"""

import numpy as np
from scipy.stats import t

# Function to calculate confidence interval for Cohen's d
def cohen_d(x, y, alpha=0.05):
    """
    Calculate Cohen's d and confidence interval for the difference between two sample means.

    Parameters:
    x (array-like): First sample data.
    y (array-like): Second sample data.
    alpha (float, optional): Significance level for the confidence interval. Default is 0.05.

    Returns:
    tuple: Tuple containing Cohen's d, lower bound of the confidence interval, and upper bound of the confidence interval.
    """

    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2  # Degrees of freedom

    # Calculate Cohen's d
    d = (np.mean(x) - np.mean(y)) / np.sqrt(((nx - 1) * np.var(x, ddof=1) + (ny - 1) * np.var(y, ddof=1)) / dof)
    
    return d

# Function to calculate confidence interval of an array of values
def confidence_interval(x, alpha=0.05):
    """
    Calculate confidence interval for an array of values.

    Parameters:
    x (array-like): Array of values.
    alpha (float, optional): Significance level for the confidence interval. Default is 0.05.

    Returns:
    tuple: Tuple containing lower bound of the confidence interval and upper bound of the confidence interval.
    """

    # Calculate mean
    mean = np.mean(x)

    # Calculate standard error
    se = np.std(x) / np.sqrt(len(x))

    # Calculate t-value for the confidence interval
    t_val = t.ppf(1 - alpha / 2, len(x) - 1)

    # Calculate margin of error
    margin_error = t_val * se

    # Calculate lower and upper bounds of the confidence interval
    lower_bound = mean - margin_error
    upper_bound = mean + margin_error

    return lower_bound, upper_bound

# List to hold the d values and the confidence intervals
d_values = []
confidence_intervals_low = []
confidence_intervals_up = []
effect = []

# Number of bootstrap iterations
n_iterations = 5000

# Perform comparison for each metric
for metric_data in [df_dsc, df_hd, df_vol]:
    for i in range(0, len(metric_data), 2):
        method_1 = metric_data[i]
        method_2 = metric_data[i + 1]
        d_values_iter = []

        # Bootstrap sampling
        for _ in range(n_iterations):
            sample_1 = np.random.choice(method_1, size=int(0.8 * len(method_1)), replace=True)
            sample_2 = np.random.choice(method_2, size=int(0.8 * len(method_2)), replace=True)

            d = cohen_d(sample_1, sample_2)

            d_values_iter.append(d)

        # Calculate mean and confidence intervals
        d_values.append(np.mean(d_values_iter))
        confidence_intervals_low.append(confidence_interval(d_values_iter))
        confidence_intervals_up.append(confidence_interval(d_values_iter))

# Measure effect
for d_value in d_values:
    if np.abs(d_value) < 0.5:
        effect.append('low')
    elif np.abs(d_value) > 0.5 and np.abs(d_value) < 0.8:
        effect.append('medium')
    else:
        effect.append('strong')

In [ ]:
# dataframe with labels, d_values and confidence intervals
df_cohen_d_bootstrap = pd.DataFrame({'labels': pair_labels, 'd_values': d_values, 'confidence_intervals_low': confidence_intervals_low, 'confidence_intervals_up': confidence_intervals_up, 'effect': effect})